<a href="https://colab.research.google.com/github/HannesKock/RaceTeam2_CHP/blob/main/Race_Team_Project_Notebook25.04.V2_CS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Race Team Project Notebook**
Names:

Matr.:

Please write down the Name of the Group member you worked on each section of code. This is necessary for grading by Studienbüro.

# **Analytics for Race 1**

- For each race day built one section with the code that you built for this race
- If you keep the same code during the next races copy the code over to the next section
- Describe why you built your analytics the way you do and interprete after each race what went well and what you want to improve on
- During motivation and interpretation try to cite online sources that you read to make better decisions (e.g., scientific articles on machine learning, blog posts, github pages)

In [ ]:
# Team members working on this code: Paula, Cedric, Hannes

First we read the CSV file"simulator_data.csv" into a pandas DataFrame for analysis. We displays the first few rows, lists the column names, provides summary statistics, and shows a concise overview of the dataset's structure.


In [ ]:
#Reading Data and summarizing contens
#https://www.datacamp.com/tutorial/pandas-read-csv
import pandas as pd

df = pd.read_csv("simulator_data.csv")
print(df.head())
print(df.columns)
print(df.describe())
print(df.info())


To see what the Data looks like we create scatter plots showing the relationship between each parameter in the dataset and lap time.

In [ ]:
# Scatterplots (Parameter vs. Lap Time)


import matplotlib.pyplot as plt
import seaborn as sns

param_cols = df.columns.drop("Lap Time")

for col in param_cols:
    plt.figure()
    sns.scatterplot(x=df[col], y=df["Lap Time"], alpha=0.3)
    plt.title(f"{col} vs. Lap Time")
    plt.show()


We noticed that the Laps within the "Simulator_Data.csv" have different lengths, therefore we divide "Lap Distance" by "Lap_time" and multiply by 3600 to get "Avg. Speed" in km/h.

Then we again generate scatter plots to visualize the relationship between each parameter (excluding "Lap Time" and "Avg. Speed") and the calculated average speed.

In [ ]:
# Scatterplots (Parameter vs. Avg. Speed)

import matplotlib.pyplot as plt
import seaborn as sns

# Durchschnittsgeschwindigkeit berechnen (falls noch nicht vorhanden)
if "Avg. Speed" not in df.columns:
    df["Avg. Speed"] = df["Lap Distance"] / (df["Lap Time"] / 3600)

# Alle Spalten außer "Lap Time" und "Avg. Speed" verwenden
param_cols = df.columns.drop(["Lap Time", "Avg. Speed"])

# Scatterplots erzeugen
for col in param_cols:
    plt.figure()
    sns.scatterplot(x=df[col], y=df["Avg. Speed"], alpha=0.3)
    plt.title(f"{col} vs. Avg. Speed")
    plt.xlabel(col)
    plt.ylabel("Avg. Speed (km/h)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

Because the scatter plots did not lead to much insights, we wanted to se how the different parameters correlate with each other. We hoped to be able to use logic to figure out how to set the car paramters.

We calculated the correlation matrix to explore how the variables relate to each other. The correlations are visualized using a heatmap to easily identify weak or strong linear relationships between parameters.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1: Load your CSV file into a pandas DataFrame
df = pd.read_csv("simulator_data.csv")  # replace with your file path

# Step 2: Calculate the correlation matrix
corr_matrix = df.corr()

# Step 3: Plot the correlation matrix
plt.figure(figsize=(20, 20))  # adjust size as needed
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", square=True, vmin=-0.05, vmax=0.05)
plt.title("Correlation Matrix")
plt.show()


The next four code blocks are first trys at using ML Models to predict and further understand the Parameters.

These attempts did not lead to much, they are still in here because they are an important step on the way to our Final Solution for this weeks race analytics.

The four Codes are:
1. first try to use a randomForrest Model and optuna to find best car parameters
2. To further understand the model parameters, we used a ML Model to create a SHAP-Diagram
3. Using the Model from the SHAP-Diagram, we try to find optimal Carparameters using optuna
4. to improve the ML Model-Parameters, we do a Grid-Search which took forever and did not lead to noticably better results.

In [ ]:
!pip install optuna
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import optuna

# === 1. Daten einlesen ===
df = pd.read_csv("simulator_data.csv")

# Ziel- und Feature-Spalten
target_col = "Lap Time"
feature_cols = df.columns.drop(target_col)

# === 2. Modell trainieren ===
X = df[feature_cols]
y = df[target_col]

# Optional: Skalieren
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Training/Test-Split (z. B. für Validierung)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# === 3. Bayesian Optimization Setup (Optuna) ===

# Gegeben: Umgebungsbedingungen
fixed_conditions = {
    'Lap Distance': 3.7,
    'Cornering': 6,
    'Inclines': 20,
    'Camber': 44,
    'Grip': 1,
    'Wind (Avg. Speed)': 97,
    'Temperature': 29,
    'Humidity': 23,
    'Air Density': 70,
    'Air Pressure': 98,
    'Wind (Gusts)': 49,
    'Altitude': 31,
    'Roughness': 49,
    'Width': 29
}

def objective(trial):
    # Optimierbare Fahrzeugparameter
    params = {
        'Rear Wing': trial.suggest_float('Rear Wing', 0.0, 500),
        'Engine': trial.suggest_float('Engine', 0.0, 500),
        'Front Wing': trial.suggest_float('Front Wing', 0.0, 500),
        'Brake Balance': trial.suggest_float('Brake Balance', 0.0, 500),
        'Differential': trial.suggest_float('Differential', 0.0, 500),
        'Suspension': trial.suggest_float('Suspension', 0.0, 500),
    }

    # Kombinieren mit festen Bedingungen
    full_input = {**fixed_conditions, **params}
    X_input = pd.DataFrame([full_input])
    X_input_scaled = scaler.transform(X_input)

    # Vorhersage durch Modell
    lap_time = model.predict(X_input_scaled)[0]
    return lap_time

# Optuna-Studie starten
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Ergebnisse
print("Beste Parameterkombination:")
for key, value in study.best_params.items():
    print(f"  {key}: {value:.4f}")
print(f"Erwartete Rundenzeit: {study.best_value:.4f} Sekunden")

In [ ]:
import shap
import numpy  as np
import xgboost as xgb
from sklearn.model_selection import train_test_split

# 1. Falls noch nicht geschehen: Durchschnittsgeschwindigkeit berechnen
if "Avg. Speed" not in df.columns:
    df["Avg. Speed"] = df["Lap Distance"] / (df["Lap Time"] / 3600)

# 2. Features & Ziel definieren
feature_cols = [
    'Lap Distance', 'Cornering', 'Inclines', 'Camber', 'Grip',
    'Wind (Avg. Speed)', 'Temperature', 'Humidity', 'Air Density',
    'Air Pressure', 'Wind (Gusts)', 'Altitude', 'Roughness', 'Width',
    'Rear Wing', 'Engine', 'Front Wing', 'Brake Balance', 'Differential',
    'Suspension'
]

X = df[feature_cols]
y = df["Avg. Speed"]

# 3. Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 4. Modell trainieren (XGBoost)
#model = xgb.XGBRegressor(n_estimators=100, max_depth=5, random_state=42)
model = xgb.XGBRegressor(colsample_bytree=1.0, learning_rate = 0.05, max_depth = 6, min_child_weight = 1, n_estimators = 300, subsample = 0.8)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)
error = np.mean(np.abs(y_test - y_hat))
print(error)

# # 5. SHAP-Werte berechnen
explainer = shap.Explainer(model)
shap_values = explainer(X_test)

# # 6. SHAP Summary Plot
shap.summary_plot(shap_values, X_test)


In [ ]:

import optuna
import numpy as np
import pandas as pd

# You can get this from df.mean().values or pick a sample row
base_input = df[feature_cols].mean().values

for fc in fixed_conditions:
    base_input[feature_cols.index(fc)] = fixed_conditions[fc]

# Indices of the features we want to optimize
optim_features = ['Rear Wing', 'Engine', 'Front Wing', 'Brake Balance', 'Differential', 'Suspension']
optim_indices = [feature_cols.index(f) for f in optim_features]

# Define bounds from your dataset (here we use min/max)
feature_bounds = {
    'Rear Wing': (1, 500),
    'Engine': (1, 500),
    'Front Wing': (1, 500),
    'Brake Balance': (1, 500),
    'Differential': (1, 500),
    'Suspension': (1, 500),
}

# Objective function for Optuna
def objective(trial):
    x = base_input.copy()

    for f in optim_features:
        val = trial.suggest_int(f, int(feature_bounds[f][0]), int(feature_bounds[f][1]))
        x[feature_cols.index(f)] = val

    pred = model.predict(np.array([x]))[0]
    return pred  # Optuna will maximize if we tell it to

# Run Optuna study
optuna.logging.disable_default_handler()
study = optuna.create_study(direction='maximize')  # use 'minimize' if lower lap time is better
study.optimize(objective, n_trials=255)


# Show results
print("Best params:")
for k, v in study.best_params.items():
    print(f"{k}: {v}")

print(f"Max predicted avg speed: {study.best_value}")


In [ ]:
#Grid Search for best Model Parameters

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import numpy as np

# Load the dataset
df = pd.read_csv("simulator_data.csv")

# Create the target variable: average speed
df['Avg Speed'] = df['Lap Distance'] / df['Lap Time']

# Drop unused columns
X = df.drop(columns=['Lap Distance', 'Lap Time', 'Avg Speed'])
y = df['Avg Speed']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define XGBoost Regressor
model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Define grid search parameters
param_grid = {
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 300],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [1, 5]
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=2,
    n_jobs=-1
)

# Fit model
grid_search.fit(X_train, y_train)

# Best model
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# Evaluate
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Test RMSE: {rmse:.4f}")


## Determining, which Parameters have the biggest impacts on Lap Times / Speeds.



This code trains an XGBoost Regressor model to predict the average speed of a car based on various track and environmental features. First, it loads the data and calculates the average speed by dividing the lap distance by the lap time. The code then preprocesses the data by removing unnecessary columns and splits it into training and testing sets. After training the model, it evaluates the feature importances to determine which variables most strongly influence the predicted average speed. **Finally, it plots a bar chart to visually display the feature importances, providing insights into the relative impact of each feature.**

In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load and preprocess
df = pd.read_csv("simulator_data.csv")
df["Avg Speed"] = df["Lap Distance"] / df["Lap Time"]

X = df.drop(columns=["Lap Distance", "Lap Time", "Avg Speed"])
y = df["Avg Speed"]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = XGBRegressor()
model.fit(X_train, y_train)

# Plot feature importance
importances = model.feature_importances_
feat_importance = pd.Series(importances, index=X.columns).sort_values(ascending=False)

feat_importance.plot(kind="bar", figsize=(12, 6), title="Feature Importance")
plt.tight_layout()
plt.show()



## Determining, which Track/Weather Parameters have the biggest impacts on each car parameter

This code performs feature importance analysis for various car setup parameters based on environmental and track/weather conditions. It uses the XGBoost Regressor to model the relationship between the track/weather variables and each car setup parameter. The data is split into training and test sets, and the model is evaluated using the R² score to measure prediction accuracy. After training, the code extracts and plots the feature importances for each car parameter, helping to identify which track/weather variables have the most significant impact on each car setup.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Load your data
df = pd.read_csv("simulator_data.csv")

# Define inputs (track/weather features) and targets (car parameters)
track_weather_features = [
    'Lap Distance', 'Cornering', 'Inclines', 'Camber', 'Grip',
    'Wind (Avg. Speed)', 'Temperature', 'Humidity', 'Air Density',
    'Air Pressure', 'Wind (Gusts)', 'Altitude', 'Roughness', 'Width'
]

car_setup_params = ['Rear Wing', 'Engine', 'Front Wing', 'Brake Balance', 'Differential', 'Suspension']

# Store feature importances for each car parameter
feature_importance_dict = {}

# Loop through car parameters
for param in car_setup_params:
    X = df[track_weather_features]
    y = df[param]

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    model = XGBRegressor(random_state=42)
    model.fit(X_train, y_train)

    # Predict and calculate R² score
    y_pred = model.predict(X_test)
    score = r2_score(y_test, y_pred)
    print(f"\n{param} - R²: {score:.3f}")

    # Get feature importances and filter by importance > 0.1
    importances = model.feature_importances_
    importance_series = pd.Series(importances, index=X.columns).sort_values(ascending=False)

    # Store importances
    feature_importance_dict[param] = importance_series

    # Display the features with importance > 0.1
    print(f"\nFor {param}, track/weather parameters with importance > 0.1:")
    for feature, importance in importance_series.items():
        if importance > 0.075:
            print(f" - {feature}: {importance:.3f}")

    # Optional: Plot feature importances
    importance_series.plot(kind='barh', title=f"Feature Importance for {param}", figsize=(8, 5))
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

In [ ]:
#Basiclly the same as the code before, but the other car parameters are also included in the models to determine the parameter importance
#This is done to see, how different car parameters might influence each other

import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Load your data
df = pd.read_csv("simulator_data.csv")

# Define inputs (track/weather features) and targets (car parameters)
track_weather_features = [
    'Lap Distance', 'Cornering', 'Inclines', 'Camber', 'Grip',
    'Wind (Avg. Speed)', 'Temperature', 'Humidity', 'Air Density',
    'Air Pressure', 'Wind (Gusts)', 'Altitude', 'Roughness', 'Width'
]

car_setup_params = ['Rear Wing', 'Engine', 'Front Wing', 'Brake Balance', 'Differential', 'Suspension']

# Store feature importances for each car parameter
feature_importance_dict = {}

# Loop through car parameters
for param in car_setup_params:
    # Include track/weather features and other car parameters (excluding the target parameter itself)
    other_car_params = [p for p in car_setup_params if p != param]
    X = df[track_weather_features + other_car_params]
    y = df[param]

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    model = XGBRegressor(random_state=42)
    model.fit(X_train, y_train)

    # Predict and calculate R² score
    y_pred = model.predict(X_test)
    score = r2_score(y_test, y_pred)
    print(f"\n{param} - R²: {score:.3f}")

    # Get feature importances and sort them
    importances = model.feature_importances_
    importance_series = pd.Series(importances, index=X.columns).sort_values(ascending=False)

    # Store importances
    feature_importance_dict[param] = importance_series

    # Display the features with importance > 0.075
    print(f"\nFor {param}, all parameters with importance > 0.075:")
    for feature, importance in importance_series.items():
        if importance > 0.075:
            print(f" - {feature}: {importance:.3f}")

    # Optional: Plot feature importances
    importance_series.plot(kind='barh', title=f"Feature Importance for {param}", figsize=(8, 5))
    plt.gca().invert_yaxis()  # Invert y-axis for better readability
    plt.tight_layout()
    plt.show()


In [ ]:
!pip install optuna

## Sequential optimization of car-parameters.

We optimize each Car-parameter in a predefined order, ensuring that each optimization builds upon the previous ones. (Order: Feature Importance (highest to lowest)(see above))

For each car parameter, we first train a predictive model for average speed using relevant track/weather variables and already optimized parameters. Then, using Optuna, we search for the optimal value of the current car parameter that maximizes the predicted average speed.

What track and wheather parameters are important for each Car-Parameter was determined before (see above)

Order of optimization and relevant track/weather parameters:

1. Engine: Grip, Altitude, Humidity, Air Density, Temperature, Air Pressure, Inclines
2. Differetial: Cornering, Width, Inclines, Grip, Temprature, Air Density
3. Rear Wing: Air Pressure, Air Density, Cornering, Inclines, Wind (Avg. Speed), Humidity, Roughness
4. Break Balance: Width, Cornering, Roughness, Temperature
5. Front Wing: Air Pressure, Cornering, Air Density, Inclines, Wind (Avg. Speed), Humidity, Wind (Gusts)
6. Suspension: Grip, Inclines, Cornering, Camber, Width, Roughness


In [ ]:
import pandas as pd
import numpy as np
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv("simulator_data.csv")
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"])*3600

# Fixed track & weather settings — user-defined!
fixed_conditions = {
    "Cornering": 6,
    "Inclines": 20,
    "Camber": 44,
    "Grip": 1,
    "Altitude": 31,
    "Roughness": 49,
    "Width": 29,
    "Temperature": 29,
    "Humidity": 23,
    "Wind (Avg. Speed)": 97,
    "Wind (Gusts)": 49,
    "Air Density": 70,
    "Air Pressure": 98,
}

# Order of optimization and relevant features
optimization_order = [
    ("Engine", ["Grip", "Altitude", "Humidity", "Air Density", "Temperature", "Air Pressure", "Inclines"]),
    ("Differential", ["Cornering", "Width", "Inclines", "Grip", "Temperature", "Air Density"]),
    ("Rear Wing", ["Air Pressure", "Air Density", "Cornering", "Inclines", "Wind (Avg. Speed)", "Humidity", "Roughness"]),
    ("Brake Balance", ["Width", "Cornering", "Roughness", "Temperature"]),
    ("Front Wing", ["Air Pressure", "Cornering", "Air Density", "Inclines", "Wind (Avg. Speed)", "Humidity", "Wind (Gusts)"]),
    ("Suspension", ["Grip", "Inclines", "Cornering", "Camber", "Width", "Roughness"]),
]

# Storage for optimized parameters
optimized_params = {}


# Optimization loop
for param, relevant_features in optimization_order:
    print(f"\n Optimizing {param}...")

    # Features for model = track/weather + already optimized + current param
    model_features = relevant_features + list(optimized_params.keys()) + [param]
    model_df = df[model_features + ["Avg Speed"]].dropna()

    X = model_df[model_features]
    y = model_df["Avg Speed"]

    # Train model
    model = XGBRegressor(random_state=42)
    model.fit(X, y)

    # Prepare fixed input for this stage
    input_row = {f: fixed_conditions[f] for f in relevant_features}
    input_row.update(optimized_params)  # Include already optimized parameters

    def objective(trial):
        trial_value = trial.suggest_int(param, 1, 500)
        row = input_row.copy()
        row[param] = trial_value
        df_input = pd.DataFrame([row])
        pred = model.predict(df_input)[0]
        return pred  # Maximizing Avg Speed

    optuna.logging.disable_default_handler()
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    print(f"Max predicted avg speed: {study.best_value}")

    best_val = study.best_params[param]
    optimized_params[param] = best_val

    print(f" Best {param}: {best_val}")

# Final output
print("\n All optimized parameters:")
for k, v in optimized_params.items():
    print(f"{k}: {v}")


In [ ]:
All optimized parameters:
Engine: 30
Differential: 1
Rear Wing: 48
Brake Balance: 3
Front Wing: 37
Suspension: 124

In [ ]:
#Same code as above, but the optimization order of the Car-Parameters is flipped.

import pandas as pd
import numpy as np
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv("simulator_data.csv")
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"])*3600

# Fixed track & weather settings — user-defined!
fixed_conditions = {
    "Cornering": 6,
    "Inclines": 20,
    "Camber": 44,
    "Grip": 1,
    "Altitude": 31,
    "Roughness": 49,
    "Width": 29,
    "Temperature": 29,
    "Humidity": 23,
    "Wind (Avg. Speed)": 97,
    "Wind (Gusts)": 49,
    "Air Density": 70,
    "Air Pressure": 98,
}

# Order of optimization and relevant features
optimization_order = [
    ("Suspension", ["Grip", "Inclines", "Cornering", "Camber", "Width", "Roughness"]),
    ("Front Wing", ["Air Pressure", "Cornering", "Air Density", "Inclines", "Wind (Avg. Speed)", "Humidity", "Wind (Gusts)"]),
    ("Brake Balance", ["Width", "Cornering", "Roughness", "Temperature"]),
    ("Rear Wing", ["Air Pressure", "Air Density", "Cornering", "Inclines", "Wind (Avg. Speed)", "Humidity", "Roughness"]),
    ("Differential", ["Cornering", "Width", "Inclines", "Grip", "Temperature", "Air Density"]),
    ("Engine", ["Grip", "Altitude", "Humidity", "Air Density", "Temperature", "Air Pressure", "Inclines"])
]

# Storage for optimized parameters
optimized_params = {}


# Optimization loop
for param, relevant_features in optimization_order:
    print(f"\n Optimizing {param}...")

    # Features for model = track/weather + already optimized + current param
    model_features = relevant_features + list(optimized_params.keys()) + [param]
    model_df = df[model_features + ["Avg Speed"]].dropna()

    X = model_df[model_features]
    y = model_df["Avg Speed"]

    # Train model
    model = XGBRegressor(random_state=42)
    model.fit(X, y)

    # Prepare fixed input for this stage
    input_row = {f: fixed_conditions[f] for f in relevant_features}
    input_row.update(optimized_params)  # Include already optimized parameters

    def objective(trial):
        trial_value = trial.suggest_int(param, 1, 500)
        row = input_row.copy()
        row[param] = trial_value
        df_input = pd.DataFrame([row])
        pred = model.predict(df_input)[0]
        return pred  # Maximizing Avg Speed

    #optuna.logging.disable_default_handler()
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=1000)
    print(f"Max predicted avg speed: {study.best_value}")

    best_val = study.best_params[param]
    optimized_params[param] = best_val

    print(f" Best {param}: {best_val}")

# Final output
print("\n All optimized parameters:")
for k, v in optimized_params.items():
    print(f"{k}: {v}")


## Race Strategy

In [ ]:
import math

# Define the tire parameters and their lap time formulas
def lap_time_super_soft(X):
    return (69.269-0.141812499999999) + 0.141812499999999 * X

def lap_time_soft(X):
    return (70.53-0.0667741935483869) + 0.0667741935483869 * X

def lap_time_medium(X):
    return (70.927-0.0670857142857143) + 0.0670857142857143 * X

def lap_time_hard(X):
    return (70.257-0.109609756097561) + 0.109609756097561 * X

# Tire data with their lifespan
tire_lifespan = {
    "super_soft": 16,
    "soft": 30,
    "medium": 35,
    "hard": 41
}

# Pit stop penalty
pit_stop_time = 30  # seconds

# Function to calculate the total race time for a given strategy
def calculate_race_time(laps, strategy):
    total_time = 0
    total_pit_stops = 0
    lap_index = 0
    lap_counter = 0

    while lap_counter < laps:
        tire, stint_laps = strategy[lap_index]

        # Ensure we don't exceed the total laps
        if lap_counter + stint_laps > laps:
            stint_laps = laps - lap_counter

        # Calculate the lap times for this stint
        lap_times = []
        for i in range(stint_laps):
            if tire == "super_soft":
                lap_times.append(lap_time_super_soft(i + 1))
            elif tire == "soft":
                lap_times.append(lap_time_soft(i + 1))
            elif tire == "medium":
                lap_times.append(lap_time_medium(i + 1))
            elif tire == "hard":
                lap_times.append(lap_time_hard(i + 1))

        total_time += sum(lap_times)  # Add the lap times of this stint
        lap_counter += stint_laps

        # If we are not at the last stint, account for a pit stop
        if lap_counter < laps:
            total_time += pit_stop_time  # Pit stop penalty
            total_pit_stops += 1

        lap_index += 1
        if lap_index >= len(strategy):
            break

    return total_time, total_pit_stops

# Function to generate possible strategies dynamically
def generate_strategies(laps):
    strategies = []
    tire_choices = ["super_soft", "soft", "medium", "hard"]

    # Generate strategies by breaking the laps into multiple stints
    for tire1 in tire_choices:
        for tire2 in tire_choices:
            for tire3 in tire_choices:
                #for tire4 in tire_choices:
                  #for tire5 in tire_choices:
                    strategy = []
                    remaining_laps = laps

                    # Create dynamic stints for each tire
                    #for tire in [tire1, tire2, tire3, tire4, tire5]:
                    #for tire in [tire1, tire2, tire3, tire4]:
                    for tire in [tire1, tire2, tire3]:
                    #for tire in [tire1, tire2]:
                        stint_laps = tire_lifespan[tire]

                        if remaining_laps > stint_laps:
                            strategy.append((tire, stint_laps))
                            remaining_laps -= stint_laps
                        else:
                            strategy.append((tire, remaining_laps))
                            break

                    if sum([stint[1] for stint in strategy]) == laps:
                        strategies.append(strategy)

    return strategies

# Function to find the best strategy
def optimize_strategy(laps):
    best_time = math.inf
    best_strategy = None

    strategies = generate_strategies(laps)

    for strategy in strategies:
        total_time, pit_stops = calculate_race_time(laps, strategy)
        if total_time < best_time:
            best_time = total_time
            best_strategy = strategy
            best_pit_stops = pit_stops

    return best_strategy, best_time, best_pit_stops


# Main function
if __name__ == "__main__":
    race_laps = 83
    best_strategy, best_time, total_pit_stops = optimize_strategy(race_laps)
    print(f"Best Strategy: {best_strategy}")
    print(f"Best Total Time: {best_time} seconds")
    print(f"Total Pit Stops: {total_pit_stops}")


In [ ]:
Best Strategy: [('super_soft', 16), ('super_soft', 16), ('super_soft', 16), ('super_soft', 16), ('soft', 19)]
Best Total Time: 5972.774387096774 seconds

Best Strategy: [('super_soft', 16), ('super_soft', 16), ('soft', 30), ('soft', 21)]
Best Total Time: 5980.742354838709 seconds

Best Strategy: [('soft', 30), ('soft', 30), ('soft', 23)]
Best Total Time: 5988.977419354838 seconds

Random Car-Parameters, to see, if optimized values are actually better than other values.

In [ ]:
from random import randint

print(f'Rear Wing {randint(1, 500)}')
print(f'Engine {randint(1, 500)}')
print(f'Front Wing {randint(1, 500)}')
print(f'Brakebalance {randint(1, 500)}')
print(f'Differential {randint(1, 500)}')
print(f'Suspension {randint(1, 500)}')



# **Analytics for Race 2**

## AllInOne Optimization

In [ ]:
# Team members working on this code: Names..

In [ ]:
!pip install optuna

In [ ]:
!pip install optuna_dashboard

### Hyperparameter Tuning for Full LGBM

In [ ]:
#https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("simulator_data.csv")
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"])*3600

# Example: Regression task
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True  # Optional but often speeds things up
    }


    # K-Fold Cross-Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    X = df.drop(columns=['Lap Distance', 'Lap Time', 'Avg Speed'])
    y = df["Avg Speed"]
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X_val, label=y_val)

        model = lgb.train(param, dtrain, num_boost_round=100,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        #rmse = mean_squared_error(y_val, preds, squared=False)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)

print("Best trial:")
print(study.best_trial.params)


In [ ]:
#Hyperparameter Tuning for two datasets

import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load both datasets
sim_df = pd.read_csv("simulator_data.csv")
prac_df = pd.read_csv("practice_dataV2_6.csv")

# Calculate target: Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0  # Adjust this ratio to control importance (10000/40 = 250)

# Merge datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Prepare features and target
X = df.drop(columns=["Avg Speed", "Lap Distance", "Lap Time","Round", "Track", "Qualifying", "Stint", "Lap", "Fuel", "Tyre Remaining", "Tyre Choice"])
y = df["Avg Speed"]
weights = df["weight"]

# Objective function for Optuna
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        w_train, w_val = weights.iloc[train_idx], weights.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train, weight=w_train)
        dvalid = lgb.Dataset(X_val, label=y_val, weight=w_val)

        model = lgb.train(param, dtrain, num_boost_round=1000,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("Best trial:")
print(study.best_trial.params)


### R^2 Test LightGBM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df.drop(columns=["Avg Speed", "Lap Distance", "Lap Time"])
y = df["Avg Speed"]


params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 242,
    'max_depth': 9,
    'learning_rate': 0.0645474493988489,
    'min_data_in_leaf': 96,
    'feature_fraction': 0.9427185636727835,
    'bagging_fraction': 0.785711983453578,
    'bagging_freq': 2,
    'lambda_l1': 0.6006835353397709,
    'lambda_l2': 1.4905819957883624,
    'objective': 'regression',
    'metric': 'l2',
    }


# K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
r2_scores = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # LightGBM Dataset
    lgb_train = lgb.Dataset(X_train, y_train)

    # Train model
    model = lgb.train(
        params,
        train_set=lgb_train,
        num_boost_round=300  )

    # Predict and evaluate
    y_pred = model.predict(X_val)
    r2 = r2_score(y_val, y_pred)
    r2_scores.append(r2)
    print(f"Fold R^2 Score: {r2:.4f}")

# Overall R^2
print(f"\nAverage R^2 Score: {np.mean(r2_scores):.4f}")


### Training LGBM-Model with Simulator_Data and optm. Hyperparameters

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df.drop(columns=["Avg Speed", "Lap Distance", "Lap Time"])
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 242,
    'max_depth': 9,
    'learning_rate': 0.0645474493988489,
    'min_data_in_leaf': 96,
    'feature_fraction': 0.9427185636727835,
    'bagging_fraction': 0.785711983453578,
    'bagging_freq': 2,
    'lambda_l1': 0.6006835353397709,
    'lambda_l2': 1.4905819957883624,
}

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("best_lgbm_model.txt")

print("Model training complete and saved to 'best_lgbm_model.txt'")


In [ ]:
#Directly trained LGBM-Model with Simulator_Data and Practice_data using optm. Hyperparameters
#Hyperparameters optm. for Simulator and Practice_data

import pandas as pd
import numpy as np
import lightgbm as lgb

# Load both datasets
sim_df = pd.read_csv("simulator_data.csv")
prac_df = pd.read_csv("practice_dataV2_6.csv")

# Compute Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0

# Make sure both have the same columns in the same order
required_cols = ["Lap Distance", "Lap Time", "Avg Speed", "weight", "Rear Wing", "Front Wing", "Engine", "Brake Balance", "Differential", "Suspension", "Cornering", "Inclines", "Camber", "Grip", "Altitude", "Roughness", "Width", "Temperature", "Humidity", "Wind (Avg. Speed)", "Wind (Gusts)", "Air Density", "Air Pressure"] + \
                [col for col in sim_df.columns if col not in ["Lap Distance", "Lap Time", "Avg Speed", "weight", "Rear Wing", "Front Wing", "Engine", "Brake Balance", "Differential", "Suspension", "Cornering", "Inclines", "Camber", "Grip", "Altitude", "Roughness", "Width", "Temperature", "Humidity", "Wind (Avg. Speed)", "Wind (Gusts)", "Air Density", "Air Pressure"]]

sim_df = sim_df[required_cols]
prac_df = prac_df[required_cols]

# Combine datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Features, target, and weights
X = df.drop(columns=["Avg Speed", "Lap Distance", "Lap Time", "weight"])
y = df["Avg Speed"]
weights = df["weight"]

# Define LightGBM dataset with weights
lgb_data = lgb.Dataset(X, label=y, weight=weights)

# Parameters (already tuned)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 74,
    'max_depth': 11,
    'learning_rate': 0.028870630893942515,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.7399238619191133,
    'bagging_fraction': 0.9203471068533383,
    'bagging_freq': 6,
    'lambda_l1': 0.025758523534949052,
    'lambda_l2': 0.9073066525703107}

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("sim_prac_lgbm_model.txt")

print("Model training complete and saved to 'sim_prac_lgbm_model.txt'")


### Improving previously traind LGBM-Model using Pracice_Data

In [ ]:
import pandas as pd
import lightgbm as lgb

# Load the original model
model = lgb.Booster(model_file="best_lgbm_model.txt")

# Load the new data
new_df = pd.read_csv("practice_dataV2_5.csv")

# Compute Avg Speed for new data
new_df["Avg Speed"] = (new_df["Lap Distance"] / new_df["Lap Time"]) * 3600

# Prepare features and target
X_new = new_df.drop(columns=["Avg Speed", "Lap Distance", "Lap Time","Round", "Track", "Qualifying", "Stint", "Lap", "Fuel", "Tyre Remaining", "Tyre Choice"])
y_new = new_df["Avg Speed"]

# Assign higher weights (e.g., 10x more important)
sample_weight = [1000] * len(y_new)

# Create new LightGBM Dataset
new_data = lgb.Dataset(X_new, label=y_new, weight=sample_weight)

# Continue training from previous model
model = lgb.train(
    params={},  # Empty here since model already knows them
    train_set=new_data,
    init_model=model,
    num_boost_round=1000,  # You can increase if needed
)

# Save updated model
model.save_model("best_lgbm_model_updated.txt")

print("Model updated with new data and saved to 'best_lgbm_model_updated.txt'")


### Used to test prediction performance against practice_data

In [ ]:
import pandas as pd
import lightgbm as lgb
import numpy as np

# Load the trained model
model = lgb.Booster(model_file="best_lgbm_model.txt")

# Load the test data
test_df = pd.read_csv("practice_data_snd_half.csv")

# Calculate actual average speed
test_df["Actual Avg Speed"] = (test_df["Lap Distance"] / test_df["Lap Time"]) * 3600

# Prepare the test features (same feature engineering as training)
X_test = test_df.drop(columns=["Actual Avg Speed", "Lap Distance", "Lap Time","Round", "Track", "Qualifying", "Stint", "Lap", "Fuel", "Tyre Remaining", "Tyre Choice"])

# Predict using the model
test_df["Predicted Avg Speed"] = model.predict(X_test)

# Calculate difference
test_df["Difference"] = test_df["Predicted Avg Speed"] - test_df["Actual Avg Speed"]

# Output the results
print(test_df[["Predicted Avg Speed", "Actual Avg Speed", "Difference"]])

# Summary statistics
mae = np.mean(np.abs(test_df["Difference"]))
rmse = np.sqrt(np.mean(test_df["Difference"] ** 2))
max_diff = np.max(np.abs(test_df["Difference"]))

print("\nSummary:")
print(f"Mean Absolute Error: {mae:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"Maximum Absolute Difference: {max_diff:.2f}")


### Parameter Optimization

In [ ]:
import pandas as pd
import lightgbm as lgb
import optuna


# Load trained model
#model = lgb.Booster(model_file="best_lgbm_model_updated.txt")
model = lgb.Booster(model_file="sim_prac_lgbm_model.txt")

# Load track/weather data (single row)
track_weather = pd.read_csv("track_weather_germany.csv")

# Define the objective function
def objective(trial):
    # Suggest car parameters
    params = {
        "Engine": trial.suggest_int("Engine", 1, 500),
        "Rear Wing": trial.suggest_int("Rear Wing", 1, 500),
        "Front Wing": trial.suggest_int("Front Wing", 1, 500),
        "Brake Balance": trial.suggest_int("Brake Balance", 1, 500),
        "Suspension": trial.suggest_int("Suspension", 1, 500),
        "Differential": trial.suggest_int("Differential", 1, 500),
    }
#Engine: 91
#Differential: 101
#RearWing: 489
#FrontWing: 274
#Suspension: 45
#BrakeBalance: 101

    # Combine with static track/weather parameters
    input_data = pd.concat([track_weather, pd.DataFrame([params])], axis=1)

    # Predict average speed
    predicted_avg_speed = model.predict(input_data)[0]

    # We want to maximize speed
    return -predicted_avg_speed

# Create study
study = optuna.create_study(direction="minimize")

# Optimize
study.optimize(objective, n_trials=100)

# Output best results
best_trial = study.best_trial
print("\nBest Parameters:")
for key, value in best_trial.params.items():
    print(f"{key}: {value}")
print(f"Predicted Avg Speed: {-best_trial.value:.2f}")

In [ ]:

Best Parameters:
Engine: 133
Rear Wing: 39
Front Wing: 67
Brake: 217
Suspension: 95
Differential: 88
Predicted Avg Speed: --187.46

Engine: 91
Differential: 101
RearWing: 489
FrontWing: 274
Suspension: 45
BrakeBalance: 101

In [ ]:
#Set the intervals around the supposedly optimal parameters


In [ ]:
import pandas as pd
import lightgbm as lgb
import optuna
import threading

# Load trained model
#model = lgb.Booster(model_file="best_lgbm_model_updated.txt")
model = lgb.Booster(model_file="best_lgbm_model.txt")

# Load track/weather data (single row)
track_weather = pd.read_csv("track_weather_germany.csv")

# Store all best results
all_best_results = []

# Optional: Start dashboard for one of the runs
def run_dashboard(study):
    run_server(study)

# Run 20 optimization loops
for run in range(1, 11):
    print(f"\n🔁 Optimization Run {run}/20")

    # Define objective function (wrapped inside loop to keep it clean)
    def objective(trial):
        params = {
            "Engine": trial.suggest_int("Engine", 85, 115),
            "Rear Wing": trial.suggest_int("Rear Wing", 499, 500),
            "Front Wing": trial.suggest_int("Front Wing", 70, 90),
            "Brake": trial.suggest_int("Brake", 85, 115),
            "Suspension": trial.suggest_int("Suspension", 20, 50),
            "Differential": trial.suggest_int("Differential", 68, 98),
        }

#Engine:         91  +-30    => 85 - 115
#Differential:  101  +-30    => 68 - 98
#RearWing:      489  +-125   => 499 - 500
#FrontWing:     274  +-125   => 70 - 90
#Suspension:     45  +-125  => 20 - 50
#BrakeBalance:  101  +-125   => 85 - 115

        input_data = pd.concat([track_weather, pd.DataFrame([params])], axis=1)
        predicted_avg_speed = model.predict(input_data)[0]
        return predicted_avg_speed  # maximize speed

    # Create a new study for each run
    study = optuna.create_study(direction="maximize")


    # Run optimization (adjust trials as needed)
    study.optimize(objective, n_trials=100)

    # Collect results
    best_trial = study.best_trial
    result = {
        "Run": run,
        "Predicted Avg Speed": best_trial.value,
        **best_trial.params
    }
    all_best_results.append(result)

# Convert to DataFrame
results_df = pd.DataFrame(all_best_results)
print("\n📊 Summary of All Runs:")
print(results_df)

# Optionally save to CSV
results_df.to_csv("optimization_results.csv", index=False)


In [ ]:
#Engine:         91  +-30    => 61 - 121
#Differential:  101  +-30    => 51 - 151
#RearWing:      489  +-125   => 364 - 500
#FrontWing:     274  +-125   => 149 - 399
#Suspension:     45  +-125  => 5 - 170
#BrakeBalance:  101  +-125   => 0 - 226

In [ ]:
📊 Summary of All Runs: (ohne update)
   Run  Predicted Avg Speed  Engine  Rear Wing  Front Wing  Brake  Suspension  \
0    1           186.038712      89        411         170    100          91
1    2           185.970222     111        369         156    210          80
2    3           186.081557     111        478         163    101          82
3    4           185.960439     105        496         154    176          97
4    5           185.998318      93        445         198    121          99
5    6           186.093750     109        427         152    107          98
6    7           186.047609      89        463         198    116          92
7    8           186.013795     110        459         149     99          88
8    9           186.048086      94        437         155    101         106
9   10           186.066058      90        388         158    114          94

   Differential
0            86
1            82
2            83
3            94
4            82
5            82
6            83
7           100
8            91
9            85

## Sequential Optimization LightGBM

Schritte:

1. Which Car-Parameters should use which track/weather-parameters

2. In what order should the Car-Parameters be optimized?

3. Write Code, that optimizes each Models Hyperparameters using Optuna and K-Fold-cross-validation.

4. Train and Save each Model with full data set (no Train/Test-Split)

5. Optimize Models using Practice_Data

6. Write Code, that sequentially optimizes Car-Parameters with existing Models







### First Step: Which Car-Parameters should use which track/weather-parameters?



Rear Wing
 - Air Density
 - Cornering
 - Air Pressure
 - Inclines
 - Wind (Avg. Speed)
 - Humidity
 - Roughness

Engine
 - Grip
 - Humidity
 - Air Density
 - Altitude
 - Temperature
 - Inclines
 - Air Pressure
 - Cornering

Front Wing
 - Cornering
 - Air Pressure
 - Air Density
 - Inclines
 - Wind (Avg. Speed)
 - Humidity
 - Wind (Gusts)

Brake Balance
 - Cornering
 - Width
 - Roughness
 - Temperature

Differential
 - Cornering
 - Width
 - Inclines
 - Grip
 - Temperature
 - Air Density

Suspension
 - Grip
 - Inclines
 - Cornering
 - Camber
 - Roughness
 - Width

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Load your data
df = pd.read_csv("simulator_data.csv")

# Define inputs (track/weather features) and targets (car parameters)
track_weather_features = [
    'Cornering', 'Inclines', 'Camber', 'Grip',
    'Wind (Avg. Speed)', 'Temperature', 'Humidity', 'Air Density',
    'Air Pressure', 'Wind (Gusts)', 'Altitude', 'Roughness', 'Width'
]

car_setup_params = ['Rear Wing', 'Engine', 'Front Wing', 'Brake Balance', 'Differential', 'Suspension']

# Store feature importances for each car parameter
feature_importance_dict = {}

# Loop through car parameters
for param in car_setup_params:
    X = df[track_weather_features]
    y = df[param]

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model
    model = LGBMRegressor(random_state=42)
    model.fit(X_train, y_train)

    # Predict and calculate R² score
    y_pred = model.predict(X_test)
    score = r2_score(y_test, y_pred)
    print(f"\n{param} - R²: {score:.3f}")

    # Get feature importances and filter by importance > 0.1
    importances = model.booster_.feature_importance(importance_type='gain')
    importance_series = pd.Series(importances, index=X.columns).sort_values(ascending=False)

    # Store importances
    feature_importance_dict[param] = importance_series

    # Display the features with importance > 0.075
    print(f"\nFor {param}, track/weather parameters with importance > 0.075:")
    for feature, importance in importance_series.items():
        if importance > 0.075:
            print(f" - {feature}: {importance:.3f}")

    # Optional: Plot feature importances
    importance_series.plot(kind='barh', title=f"Feature Importance for {param}", figsize=(8, 5))
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()


### Second Step: In what order should the Car-Parameters be optimized?

1. Engine
2. Differential
3. Rear Wing
4. Front Wing
5. Suspension
6. Break Balance

In [ ]:
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv("simulator_data.csv")
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Define X and y
X = df.drop(columns=['Lap Distance', 'Lap Time', 'Avg Speed'])
y = df["Avg Speed"]

# Set the hyperparameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 242,
    'max_depth': 9,
    'learning_rate': 0.0645474493988489,
    'min_data_in_leaf': 96,
    'feature_fraction': 0.9427185636727835,
    'bagging_fraction': 0.785711983453578,
    'bagging_freq': 2,
    'lambda_l1': 0.6006835353397709,
    'lambda_l2': 1.4905819957883624,
    'objective': 'regression',
    'metric': 'l2',
}

# Train the model
train_data = lgb.Dataset(X, label=y)
model = lgb.train(params,
                  train_data,
                  num_boost_round=100)

# Get feature importance by 'split' and 'gain'
importance_split = model.feature_importance(importance_type='split')
importance_gain = model.feature_importance(importance_type='gain')

# Sort feature importance by 'split'
sorted_split_idx = importance_split.argsort()[::1]
sorted_split_importance = importance_split[sorted_split_idx]
sorted_split_features = X.columns[sorted_split_idx]

# Sort feature importance by 'gain'
sorted_gain_idx = importance_gain.argsort()[::1]
sorted_gain_importance = importance_gain[sorted_gain_idx]
sorted_gain_features = X.columns[sorted_gain_idx]

# Plot feature importance by 'split'
plt.figure(figsize=(12, 6))
plt.barh(sorted_split_features, sorted_split_importance , color='red')
plt.title('Feature Importance by Split')
plt.xlabel('Number of Splits')
plt.ylabel('Features')
plt.show()

# Plot feature importance by 'gain'
plt.figure(figsize=(12, 6))
plt.barh(sorted_gain_features, sorted_gain_importance, color='blue')
plt.title('Feature Importance by Gain')
plt.xlabel('Gain')
plt.ylabel('Features')
plt.show()


### Third Step: Write Code, that optimizes each Models Hyperparameters using Optuna and K-Fold-cross-validation.

In [ ]:
#Optimized Hyperparameters:
Engine:
{'num_leaves': 281, 'max_depth': 11, 'learning_rate': 0.04083240077095524, 'min_data_in_leaf': 96, 'feature_fraction': 0.9534759457760267, 'bagging_fraction': 0.6637656869307713, 'bagging_freq': 7, 'lambda_l1': 0.48486124811843856, 'lambda_l2': 1.0469435993282974}
Differential:
{'num_leaves': 126, 'max_depth': 8, 'learning_rate': 0.05725939263903659, 'min_data_in_leaf': 96, 'feature_fraction': 0.9048274481436259, 'bagging_fraction': 0.6871315558035545, 'bagging_freq': 1, 'lambda_l1': 2.9809700145880162, 'lambda_l2': 2.893530172381254}
Rear Wing:
{'num_leaves': 41, 'max_depth': 14, 'learning_rate': 0.07379289439034513, 'min_data_in_leaf': 14, 'feature_fraction': 0.9805583874353474, 'bagging_fraction': 0.757494844528942, 'bagging_freq': 1, 'lambda_l1': 3.3280185770912905, 'lambda_l2': 4.668868493827121}
Front Wing:
{'num_leaves': 228, 'max_depth': 6, 'learning_rate': 0.10991909073754202, 'min_data_in_leaf': 94, 'feature_fraction': 0.954670793846081, 'bagging_fraction': 0.7659843231590522, 'bagging_freq': 1, 'lambda_l1': 3.8616883301361553, 'lambda_l2': 4.896984209036385}
Suspension:
{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.14051851191638579, 'min_data_in_leaf': 75, 'feature_fraction': 0.999888882811791, 'bagging_fraction': 0.7009625088481276, 'bagging_freq': 1, 'lambda_l1': 4.8851994495913145, 'lambda_l2': 3.6824821872767783}
Break Balance:
{'num_leaves': 27, 'max_depth': 11, 'learning_rate': 0.18696777761444966, 'min_data_in_leaf': 73, 'feature_fraction': 0.964098272670725, 'bagging_fraction': 0.8852821315081366, 'bagging_freq': 10, 'lambda_l1': 3.486399193949678, 'lambda_l2': 3.7053586457221366}

#### HP-optimization code for each Car-Param.

In [ ]:
#Hyperparameter Optimization Engine
#https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("simulator_data.csv")
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"])*3600

# Example: Regression task
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True  # Optional but often speeds things up
    }


    # K-Fold Cross-Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    X = df[['Engine', 'Grip', 'Humidity', 'Air Density', 'Altitude', 'Temperature', 'Inclines', 'Air Pressure' , 'Cornering']]
    y = df["Avg Speed"]
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X_val, label=y_val)

        model = lgb.train(param, dtrain, num_boost_round=100,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        #rmse = mean_squared_error(y_val, preds, squared=False)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)

print("Best trial:")
print(study.best_trial.params)


In [ ]:
#Hyperparameter Optimization Differential
#https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("simulator_data.csv")
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"])*3600

# Example: Regression task
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True  # Optional but often speeds things up
    }


    # K-Fold Cross-Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    X = df[['Differential', 'Engine', 'Cornering', 'Width', 'Inclines', 'Grip', 'Temperature', 'Air Density']]
    y = df["Avg Speed"]
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X_val, label=y_val)

        model = lgb.train(param, dtrain, num_boost_round=100,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        #rmse = mean_squared_error(y_val, preds, squared=False)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)

print("Best trial:")
print(study.best_trial.params)


In [ ]:
#Hyperparameter Optimization Rear Wing
#https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("simulator_data.csv")
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"])*3600

# Example: Regression task
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True  # Optional but often speeds things up
    }


    # K-Fold Cross-Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    X = df[['Rear Wing', 'Differential', 'Engine', 'Air Density', 'Cornering', 'Air Pressure' , 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Roughness']]
    y = df["Avg Speed"]
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X_val, label=y_val)

        model = lgb.train(param, dtrain, num_boost_round=100,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        #rmse = mean_squared_error(y_val, preds, squared=False)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)

print("Best trial:")
print(study.best_trial.params)


In [ ]:
#Hyperparameter Optimization Front Wing
#https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("simulator_data.csv")
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"])*3600

# Example: Regression task
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True  # Optional but often speeds things up
    }


    # K-Fold Cross-Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    X = df[['Front Wing', 'Rear Wing', 'Differential', 'Engine', 'Cornering', 'Air Pressure', 'Air Density', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Wind (Gusts)']]
    y = df["Avg Speed"]
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X_val, label=y_val)

        model = lgb.train(param, dtrain, num_boost_round=100,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        #rmse = mean_squared_error(y_val, preds, squared=False)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)

print("Best trial:")
print(study.best_trial.params)


In [ ]:
#Hyperparameter Optimization Suspension
#https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("simulator_data.csv")
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"])*3600

# Example: Regression task
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True  # Optional but often speeds things up
    }


    # K-Fold Cross-Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    X = df[['Suspension', 'Front Wing', 'Rear Wing', 'Differential', 'Engine', 'Grip', 'Inclines', 'Cornering', 'Camber', 'Roughness', 'Width']]
    y = df["Avg Speed"]
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X_val, label=y_val)

        model = lgb.train(param, dtrain, num_boost_round=100,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        #rmse = mean_squared_error(y_val, preds, squared=False)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)

print("Best trial:")
print(study.best_trial.params)


In [ ]:
#Hyperparameter Optimization Brake Balance
#https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("simulator_data.csv")
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"])*3600

# Example: Regression task
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True  # Optional but often speeds things up
    }


    # K-Fold Cross-Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    X = df[['Brake Balance', 'Suspension', 'Front Wing', 'Rear Wing', 'Differential', 'Engine','Cornering', 'Width', 'Roughness', 'Temperature']]
    y = df["Avg Speed"]
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X_val, label=y_val)

        model = lgb.train(param, dtrain, num_boost_round=100,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        #rmse = mean_squared_error(y_val, preds, squared=False)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)

print("Best trial:")
print(study.best_trial.params)


### Fourth Step: Train and Save each Model with full data set (no Train/Test-Split)

In [ ]:
Brake Balance: 	X = df[['Brake_Balance', 'Suspension', 'Front_Wing', 'Rear_Wing', 'Differential', 'Engine','Cornering', 'Width', 'Roughness', 'Temperature']]
Suspension: 	X = df[['Suspension', 'Front_Wing', 'Rear_Wing', 'Differential', 'Engine', 'Grip', 'Inclines', 'Cornering', 'Camber', 'Roughness', 'Width']]
Front Wing: 	X = df[['Front_Wing', 'Rear_Wing', 'Differential', 'Engine', 'Cornering', 'Air Pressure', 'Air Density', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Wind (Gusts)']]
Rear Wing: 	X = df[['Rear_Wing', 'Differential', 'Engine', 'Air Density', 'Cornering', 'Air Pressure' , 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Roughness']]
Differential: 	X = df[['Differential', 'Engine', 'Cornering', 'Width', 'Inclines', 'Grip', 'Temperature', 'Air Density']]
Engine: 	X = df[['Engine', 'Grip', 'Humidity', 'Air Density', 'Altitude', 'Temperature', 'Inclines', 'Air Pressure' , 'Cornering']]

In [ ]:
#Optimized Hyperparameters:
Engine:
{'num_leaves': 281, 'max_depth': 11, 'learning_rate': 0.04083240077095524, 'min_data_in_leaf': 96, 'feature_fraction': 0.9534759457760267, 'bagging_fraction': 0.6637656869307713, 'bagging_freq': 7, 'lambda_l1': 0.48486124811843856, 'lambda_l2': 1.0469435993282974}
Differential:
{'num_leaves': 126, 'max_depth': 8, 'learning_rate': 0.05725939263903659, 'min_data_in_leaf': 96, 'feature_fraction': 0.9048274481436259, 'bagging_fraction': 0.6871315558035545, 'bagging_freq': 1, 'lambda_l1': 2.9809700145880162, 'lambda_l2': 2.893530172381254}
Rear Wing:
{'num_leaves': 41, 'max_depth': 14, 'learning_rate': 0.07379289439034513, 'min_data_in_leaf': 14, 'feature_fraction': 0.9805583874353474, 'bagging_fraction': 0.757494844528942, 'bagging_freq': 1, 'lambda_l1': 3.3280185770912905, 'lambda_l2': 4.668868493827121}
Front Wing:
{'num_leaves': 228, 'max_depth': 6, 'learning_rate': 0.10991909073754202, 'min_data_in_leaf': 94, 'feature_fraction': 0.954670793846081, 'bagging_fraction': 0.7659843231590522, 'bagging_freq': 1, 'lambda_l1': 3.8616883301361553, 'lambda_l2': 4.896984209036385}
Suspension:
{'num_leaves': 31, 'max_depth': 10, 'learning_rate': 0.14051851191638579, 'min_data_in_leaf': 75, 'feature_fraction': 0.999888882811791, 'bagging_fraction': 0.7009625088481276, 'bagging_freq': 1, 'lambda_l1': 4.8851994495913145, 'lambda_l2': 3.6824821872767783}
Break Balance:
{'num_leaves': 27, 'max_depth': 11, 'learning_rate': 0.18696777761444966, 'min_data_in_leaf': 73, 'feature_fraction': 0.964098272670725, 'bagging_fraction': 0.8852821315081366, 'bagging_freq': 10, 'lambda_l1': 3.486399193949678, 'lambda_l2': 3.7053586457221366}

#### Code to build Model for each Car-Param.

**"_newnames" explained:**
because of how the following code works, the coloums in the Data need to have names, that fit a certain scheme. The coloums cant have blank spaces between words, the blank spaces can also not be replaced with underscores (_).
Also the trained Models need to have the exact same name, as their corresponding car-parameter.

Rear Wing: RearWing

Front Wing: FrontWing

Brake: BrakeBalance

This needs to be done for Simulator and Practice_Data.

In [ ]:
#Engine (Hyperparameters and X set)
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data_newnames.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df[['Engine', 'Grip', 'Humidity', 'Air Density', 'Altitude', 'Temperature', 'Inclines', 'Air Pressure' , 'Cornering']]
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 281,
    'max_depth': 11,
    'learning_rate': 0.04083240077095524,
    'min_data_in_leaf': 96,
    'feature_fraction': 0.9534759457760267,
    'bagging_fraction': 0.6637656869307713,
    'bagging_freq': 7,
    'lambda_l1': 0.48486124811843856,
    'lambda_l2': 1.0469435993282974
}

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("Engine_lgbm_model.txt")

print("Model training complete and saved to 'Engine_lgbm_model.txt'")


In [ ]:
#Differential (Hyperparameters and X set)
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data_newnames.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df[['Differential', 'Engine', 'Cornering', 'Width', 'Inclines', 'Grip', 'Temperature', 'Air Density']]
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 126,
    'max_depth': 8,
    'learning_rate': 0.05725939263903659,
    'min_data_in_leaf': 96,
    'feature_fraction': 0.9048274481436259,
    'bagging_fraction': 0.6871315558035545,
    'bagging_freq': 1,
    'lambda_l1': 2.9809700145880162,
    'lambda_l2': 2.893530172381254
    }


# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("Differential_lgbm_model.txt")

print("Model training complete and saved to 'Differential_lgbm_model.txt'")


In [ ]:
#Rear Wing (Hyperparameters and X set)
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data_newnames.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df[['RearWing', 'Differential', 'Engine', 'Air Density', 'Cornering', 'Air Pressure' , 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Roughness']]
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 41,
    'max_depth': 14,
    'learning_rate': 0.07379289439034513,
    'min_data_in_leaf': 14,
    'feature_fraction': 0.9805583874353474,
    'bagging_fraction': 0.757494844528942,
    'bagging_freq': 1,
    'lambda_l1': 3.3280185770912905,
    'lambda_l2': 4.668868493827121
    }

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("RearWing_lgbm_model.txt")

print("Model training complete and saved to 'RearWing_lgbm_model.txt'")


In [ ]:
#Front Wing (hyperparameters and X set)
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data_newnames.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df[['FrontWing', 'RearWing', 'Differential', 'Engine', 'Cornering', 'Air Pressure', 'Air Density', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Wind (Gusts)']]
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 228,
    'max_depth': 6,
    'learning_rate': 0.10991909073754202,
    'min_data_in_leaf': 94,
    'feature_fraction': 0.954670793846081,
    'bagging_fraction': 0.7659843231590522,
    'bagging_freq': 1, 'lambda_l1': 3.8616883301361553,
    'lambda_l2': 4.896984209036385
    }


# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("FrontWing_lgbm_model.txt")

print("Model training complete and saved to 'FrontWing_lgbm_model.txt'")


In [ ]:
#Suspension
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data_newnames.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df[['Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine', 'Grip', 'Inclines', 'Cornering', 'Camber', 'Roughness', 'Width']]
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'max_depth': 10,
    'learning_rate': 0.14051851191638579,
    'min_data_in_leaf': 75,
    'feature_fraction': 0.999888882811791,
    'bagging_fraction': 0.7009625088481276,
    'bagging_freq': 1,
    'lambda_l1': 4.8851994495913145,
    'lambda_l2': 3.6824821872767783
}

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("Suspension_lgbm_model.txt")

print("Model training complete and saved to 'Suspension_lgbm_model.txt'")


In [ ]:
#Brake Balance (hyperparameters and X set)
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data_newnames.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df[['BrakeBalance', 'Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine','Cornering', 'Width', 'Roughness', 'Temperature']]
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 27,
    'max_depth': 11,
    'learning_rate': 0.18696777761444966,
    'min_data_in_leaf': 73,
    'feature_fraction': 0.964098272670725,
    'bagging_fraction': 0.8852821315081366,
    'bagging_freq': 10,
    'lambda_l1': 3.486399193949678,
    'lambda_l2': 3.7053586457221366
}

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("BrakeBalance_lgbm_model.txt")

print("Model training complete and saved to 'BrakeBalance_lgbm_model.txt'")


### Fifth Step: Optimize Models using Practice_Data

In [ ]:
import pandas as pd
import lightgbm as lgb

# Model names
model_names = [
    "Engine_lgbm_model.txt",
    "Differential_lgbm_model.txt",
    "RearWing_lgbm_model.txt",
    "FrontWing_lgbm_model.txt",
    "Suspension_lgbm_model.txt",
    "BrakeBalance_lgbm_model.txt"
]

# Corresponding feature sets
feature_sets = {
    "Engine": ['Engine', 'Grip', 'Humidity', 'Air Density', 'Altitude', 'Temperature', 'Inclines', 'Air Pressure', 'Cornering'],
    "Differential": ['Differential', 'Engine', 'Cornering', 'Width', 'Inclines', 'Grip', 'Temperature', 'Air Density'],
    "RearWing": ['RearWing', 'Differential', 'Engine', 'Air Density', 'Cornering', 'Air Pressure', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Roughness'],
    "FrontWing": ['FrontWing', 'RearWing', 'Differential', 'Engine', 'Cornering', 'Air Pressure', 'Air Density', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Wind (Gusts)'],
    "Suspension": ['Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine', 'Grip', 'Inclines', 'Cornering', 'Camber', 'Roughness', 'Width'],
    "BrakeBalance": ['BrakeBalance', 'Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine', 'Cornering', 'Width', 'Roughness', 'Temperature']
}

# Load new data
df = pd.read_csv("practice_data_newnamesV2_5.csv")

# Compute Avg Speed for new data
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Target variable
y_new = df["Avg Speed"]

# Sample weight
sample_weight = [1000] * len(y_new)

# Training each model with its feature set
for model_file in model_names:
    model_key = model_file.split("_")[0]  # Extract the prefix e.g., "Engine"
    features = feature_sets[model_key]

    print(f"\nUpdating {model_file} with features: {features}")

    # Prepare feature matrix
    X_new = df[features]

    # Load existing model
    model = lgb.Booster(model_file=model_file)

    # Create dataset
    new_data = lgb.Dataset(X_new, label=y_new, weight=sample_weight)

    # Continue training
    updated_model = lgb.train(
        params={},
        train_set=new_data,
        init_model=model,
        num_boost_round=1000
    )

    # Save updated model
    updated_file = model_file.replace(".txt", "_updated.txt")
    updated_model.save_model(updated_file)

    print(f"Model saved to {updated_file}")

print("\nAll models updated successfully.")


### Sixth Step: Write Code, that sequentially optimizes Car-Parameters with existing Models

In [ ]:
!pip install optuna

In [ ]:
import pandas as pd
import lightgbm as lgb
import optuna

# Ordered list of parameters to optimize
params_to_optimize = ["Engine", "Differential", "RearWing", "FrontWing", "Suspension", "BrakeBalance"]

# Corresponding feature sets for each model
feature_sets = {
    "Engine": ['Engine', 'Grip', 'Humidity', 'Air Density', 'Altitude', 'Temperature', 'Inclines', 'Air Pressure', 'Cornering'],
    "Differential": ['Differential', 'Engine', 'Cornering', 'Width', 'Inclines', 'Grip', 'Temperature', 'Air Density'],
    "RearWing": ['RearWing', 'Differential', 'Engine', 'Air Density', 'Cornering', 'Air Pressure', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Roughness'],
    "FrontWing": ['FrontWing', 'RearWing', 'Differential', 'Engine', 'Cornering', 'Air Pressure', 'Air Density', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Wind (Gusts)'],
    "Suspension": ['Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine', 'Grip', 'Inclines', 'Cornering', 'Camber', 'Roughness', 'Width'],
    "BrakeBalance": ['BrakeBalance', 'Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine', 'Cornering', 'Width', 'Roughness', 'Temperature']
}

# Load static track & weather data
base_data = pd.read_csv("track_weather_germany.csv")

# This will hold the best parameter values as we optimize them
optimized_params = {}

# Begin sequential optimization
for car_part in params_to_optimize:
    print(f"\n🔧 Optimizing {car_part}...")

    model_path = f"{car_part}_lgbm_model_updated.txt"
    model = lgb.Booster(model_file=model_path)
    features = feature_sets[car_part]

    def objective(trial):
        # Set the current parameter being optimized

        params = {
            "Engine": trial.suggest_int("Engine", 85, 115),
            "RearWing": trial.suggest_int("RearWing", 499, 500),
            "FrontWing": trial.suggest_int("FrontWing", 70, 90),
            "BrakeBalance": trial.suggest_int("BrakeBalance", 85, 115),
            "Suspension": trial.suggest_int("Suspension", 20, 50),
            "Differential": trial.suggest_int("Differential", 68, 98),}

        current_value = params[car_part]

        # Create a single row input with all needed features
        input_data = base_data.copy()

        # Add previously optimized params
        for param, value in optimized_params.items():
            input_data[param] = value

        # Add current trial value
        input_data[car_part] = current_value

        # If any missing feature, fill with 0 or a safe default
        for col in features:
            if col not in input_data.columns:
                input_data[col] = 0

        # Ensure correct order of features
        X = input_data[features]

        # Predict Avg Speed
        avg_speed = model.predict(X)[0]
        return avg_speed  # Optuna maximizes this

    # Run Optuna
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=300, show_progress_bar=True)

    best_value = study.best_params[car_part]
    optimized_params[car_part] = best_value
    print(f"✅ Best {car_part}: {best_value}")

# Final results
print("\n🎯 Final Optimized Parameters:")
for k, v in optimized_params.items():
    print(f"{k}: {v}")


In [ ]:
477	166	89	97	83	35

🎯 Final Optimized Parameters: (model without update)
Engine: 91
Differential: 101
RearWing: 489
FrontWing: 274
Suspension: 45
BrakeBalance: 101

🎯 Final Optimized Parameters:(model with update)
Engine: 32
Differential: 1
RearWing: 43
FrontWing: 68
Suspension: 29
BrakeBalance: 268

## Race Strategy

In [ ]:
import math

# Define the tire parameters and their lap time formulas
def lap_time_super_soft(X):
    return (67.1830587725926) + 0.18352607407407 * X

def lap_time_soft(X):
    return (67.5640573104347) + 0.140194202898552 * X

def lap_time_medium(X):
    return (68.2222646857471) + 0.142527494252874 * X

def lap_time_hard(X):
    return (68.9335449948148) + 0.108381851851852 * X

# Tire data with their lifespan
tire_lifespan = {
    "super_soft": 9,
    "soft": 23,
    "medium": 29,
    "hard": 36
}

# Pit stop penalty
pit_stop_time = 30  # seconds

# Function to calculate the total race time for a given strategy
def calculate_race_time(laps, strategy):
    total_time = 0
    total_pit_stops = 0
    lap_index = 0
    lap_counter = 0

    while lap_counter < laps:
        tire, stint_laps = strategy[lap_index]

        # Ensure we don't exceed the total laps
        if lap_counter + stint_laps > laps:
            stint_laps = laps - lap_counter

        # Calculate the lap times for this stint
        lap_times = []
        for i in range(stint_laps):
            if tire == "super_soft":
                lap_times.append(lap_time_super_soft(i + 1))
            elif tire == "soft":
                lap_times.append(lap_time_soft(i + 1))
            elif tire == "medium":
                lap_times.append(lap_time_medium(i + 1))
            elif tire == "hard":
                lap_times.append(lap_time_hard(i + 1))

        total_time += sum(lap_times)  # Add the lap times of this stint
        lap_counter += stint_laps

        # If we are not at the last stint, account for a pit stop
        if lap_counter < laps:
            total_time += pit_stop_time  # Pit stop penalty
            total_pit_stops += 1

        lap_index += 1
        if lap_index >= len(strategy):
            break

    return total_time, total_pit_stops

# Function to generate possible strategies dynamically
def generate_strategies(laps):
    strategies = []
    tire_choices = ["super_soft", "soft", "medium", "hard"]

    # Generate strategies by breaking the laps into multiple stints
    for tire1 in tire_choices:
        for tire2 in tire_choices:
            for tire3 in tire_choices:
                for tire4 in tire_choices:
                  for tire5 in tire_choices:
                    strategy = []
                    remaining_laps = laps

                    # Create dynamic stints for each tire
                    for tire in [tire1, tire2, tire3, tire4, tire5]:
                    #for tire in [tire1, tire2, tire3, tire4]:
                    #for tire in [tire1, tire2, tire3]:
                    #for tire in [tire1, tire2]:
                        stint_laps = tire_lifespan[tire]

                        if remaining_laps > stint_laps:
                            strategy.append((tire, stint_laps))
                            remaining_laps -= stint_laps
                        else:
                            strategy.append((tire, remaining_laps))
                            break

                    if sum([stint[1] for stint in strategy]) == laps:
                        strategies.append(strategy)

    return strategies

# Function to find the best strategy
def optimize_strategy(laps):
    best_time = math.inf
    best_strategy = None

    strategies = generate_strategies(laps)

    for strategy in strategies:
        total_time, pit_stops = calculate_race_time(laps, strategy)
        if total_time < best_time:
            best_time = total_time
            best_strategy = strategy
            best_pit_stops = pit_stops

    return best_strategy, best_time, best_pit_stops


# Main function
if __name__ == "__main__":
    race_laps = 85
    best_strategy, best_time, total_pit_stops = optimize_strategy(race_laps)
    print(f"Best Strategy: {best_strategy}")
    print(f"Best Total Time: {best_time} seconds")
    print(f"Total Pit Stops: {total_pit_stops}")

In [ ]:
Best Strategy: [('soft', 23), ('soft', 23), ('soft', 23), ('soft', 14)]
Best Total Time: 5828.617948070432 seconds
Total Pit Stops: 3

# **Analytics for Race 3**

In [ ]:
# Team members working on this code: Names..

## Sequential Optimization

In [ ]:
!pip install optuna

### Hyperparameter Optimization Sim Prac Combo

In [ ]:
#Engine SimPrac Hyperparameter Optimization
import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load both datasets
sim_df = pd.read_csv("simulator_data.csv")
prac_df = pd.read_csv("practice_dataV3_3.csv")

# Calculate target: Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0  # Adjust this ratio to control importance (10000/40 = 250)

# Merge datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Example: Regression task
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True  # Optional but often speeds things up
    }


    # K-Fold Cross-Validation
    X = df[['Engine', 'Grip', 'Humidity', 'Air Density', 'Altitude', 'Temperature', 'Inclines', 'Air Pressure' , 'Cornering']]
    y = df["Avg Speed"]
    weights = df["weight"]

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        w_train, w_val = weights.iloc[train_idx], weights.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train, weight=w_train)
        dvalid = lgb.Dataset(X_val, label=y_val, weight=w_val)

        model = lgb.train(param, dtrain, num_boost_round=1000,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=250)

print("Best trial:")
print(study.best_trial.params)


In [ ]:
#Differential SimPrac Hyperparameter Optimization
import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load both datasets
sim_df = pd.read_csv("simulator_data.csv")
prac_df = pd.read_csv("practice_dataV3_3.csv")

# Calculate target: Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0  # Adjust this ratio to control importance (10000/40 = 250)

# Merge datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Example: Regression task
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True  # Optional but often speeds things up
    }


    # K-Fold Cross-Validation
    X = df[['Differential', 'Engine', 'Cornering', 'Width', 'Inclines', 'Grip', 'Temperature', 'Air Density']]
    y = df["Avg Speed"]
    weights = df["weight"]

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        w_train, w_val = weights.iloc[train_idx], weights.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train, weight=w_train)
        dvalid = lgb.Dataset(X_val, label=y_val, weight=w_val)

        model = lgb.train(param, dtrain, num_boost_round=1000,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=250)

print("Best trial:")
print(study.best_trial.params)


In [ ]:
#Rear Wing SimPrac Hyperparameter Optimization
import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load both datasets
sim_df = pd.read_csv("simulator_data.csv")
prac_df = pd.read_csv("practice_dataV3_3.csv")

# Calculate target: Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0  # Adjust this ratio to control importance (10000/40 = 250)

# Merge datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Example: Regression task
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True  # Optional but often speeds things up
    }


    # K-Fold Cross-Validation
    X = df[['Rear Wing', 'Differential', 'Engine', 'Air Density', 'Cornering', 'Air Pressure' , 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Roughness']]
    y = df["Avg Speed"]
    weights = df["weight"]

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        w_train, w_val = weights.iloc[train_idx], weights.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train, weight=w_train)
        dvalid = lgb.Dataset(X_val, label=y_val, weight=w_val)

        model = lgb.train(param, dtrain, num_boost_round=1000,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=250)

print("Best trial:")
print(study.best_trial.params)


In [ ]:
#Front Wing SimPrac Hyperparameter Optimization
import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load both datasets
sim_df = pd.read_csv("simulator_data.csv")
prac_df = pd.read_csv("practice_dataV3_3.csv")

# Calculate target: Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0  # Adjust this ratio to control importance (10000/40 = 250)

# Merge datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Example: Regression task
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True  # Optional but often speeds things up
    }


    # K-Fold Cross-Validation
    X = df[['Front Wing', 'Rear Wing', 'Differential', 'Engine', 'Cornering', 'Air Pressure', 'Air Density', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Wind (Gusts)']]
    y = df["Avg Speed"]
    weights = df["weight"]

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        w_train, w_val = weights.iloc[train_idx], weights.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train, weight=w_train)
        dvalid = lgb.Dataset(X_val, label=y_val, weight=w_val)

        model = lgb.train(param, dtrain, num_boost_round=1000,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=250)

print("Best trial:")
print(study.best_trial.params)


In [ ]:
#Suspension SimPrac Hyperparameter Optimization
import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load both datasets
sim_df = pd.read_csv("simulator_data.csv")
prac_df = pd.read_csv("practice_dataV3_3.csv")

# Calculate target: Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0  # Adjust this ratio to control importance (10000/40 = 250)

# Merge datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Example: Regression task
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True  # Optional but often speeds things up
    }


    # K-Fold Cross-Validation
    X = df[['Suspension', 'Front Wing', 'Rear Wing', 'Differential', 'Engine', 'Grip', 'Inclines', 'Cornering', 'Camber', 'Roughness', 'Width']]
    y = df["Avg Speed"]
    weights = df["weight"]

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        w_train, w_val = weights.iloc[train_idx], weights.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train, weight=w_train)
        dvalid = lgb.Dataset(X_val, label=y_val, weight=w_val)

        model = lgb.train(param, dtrain, num_boost_round=1000,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=250)

print("Best trial:")
print(study.best_trial.params)


In [ ]:
#Brake Balance SimPrac Hyperparameter Optimization
import optuna
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Load both datasets
sim_df = pd.read_csv("simulator_data.csv")
prac_df = pd.read_csv("practice_dataV3_3.csv")

# Calculate target: Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0  # Adjust this ratio to control importance (10000/40 = 250)

# Merge datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Example: Regression task
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2, log=True),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
        'force_col_wise': True  # Optional but often speeds things up
    }


    # K-Fold Cross-Validation
    X = df[['Brake Balance', 'Suspension', 'Front Wing', 'Rear Wing', 'Differential', 'Engine','Cornering', 'Width', 'Roughness', 'Temperature']]
    y = df["Avg Speed"]
    weights = df["weight"]

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        w_train, w_val = weights.iloc[train_idx], weights.iloc[val_idx]

        dtrain = lgb.Dataset(X_train, label=y_train, weight=w_train)
        dvalid = lgb.Dataset(X_val, label=y_val, weight=w_val)

        model = lgb.train(param, dtrain, num_boost_round=1000,
                          valid_sets=[dvalid],
                          callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)])

        preds = model.predict(X_val)
        rmse = np.sqrt(((y_val - preds)**2).mean())
        scores.append(rmse)

    return np.mean(scores)

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

print("Best trial:")
print(study.best_trial.params)


In [ ]:
Brake Balance: 	X = df[['Brake_Balance', 'Suspension', 'Front_Wing', 'Rear_Wing', 'Differential', 'Engine','Cornering', 'Width', 'Roughness', 'Temperature']]
Suspension: 	X = df[['Suspension', 'Front_Wing', 'Rear_Wing', 'Differential', 'Engine', 'Grip', 'Inclines', 'Cornering', 'Camber', 'Roughness', 'Width']]
Front Wing: 	X = df[['Front_Wing', 'Rear_Wing', 'Differential', 'Engine', 'Cornering', 'Air Pressure', 'Air Density', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Wind (Gusts)']]
Rear Wing: 	X = df[['Rear_Wing', 'Differential', 'Engine', 'Air Density', 'Cornering', 'Air Pressure' , 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Roughness']]
Differential: 	X = df[['Differential', 'Engine', 'Cornering', 'Width', 'Inclines', 'Grip', 'Temperature', 'Air Density']]
Engine: 	X = df[['Engine', 'Grip', 'Humidity', 'Air Density', 'Altitude', 'Temperature', 'Inclines', 'Air Pressure' , 'Cornering']]

### Training and saving LGBM-Models for each Car-Parameter (SimPrac Combo)

In [ ]:
Best Engine trial:
{'num_leaves': 46, 'max_depth': 5, 'learning_rate': 0.033632492994555285, 'min_data_in_leaf': 10, 'feature_fraction': 0.9292816238478901, 'bagging_fraction': 0.6029213489987132, 'bagging_freq': 1, 'lambda_l1': 4.9435895890172095, 'lambda_l2': 1.5835241201372334}

Best Differential trial:
{'num_leaves': 120, 'max_depth': 4, 'learning_rate': 0.06188740190651351, 'min_data_in_leaf': 10, 'feature_fraction': 0.8516027132719757, 'bagging_fraction': 0.7665333052378347, 'bagging_freq': 2, 'lambda_l1': 4.842140375323366, 'lambda_l2': 0.8572408057266877}

Best Rear Wing trial:
{'num_leaves': 234, 'max_depth': 8, 'learning_rate': 0.009001605992864, 'min_data_in_leaf': 12, 'feature_fraction': 0.7155500136774263, 'bagging_fraction': 0.9782633351035283, 'bagging_freq': 3, 'lambda_l1': 4.0691300429761625, 'lambda_l2': 0.4270601685583596}

Best Front Win trail:
{'num_leaves': 75, 'max_depth': 15, 'learning_rate': 0.008393206404444, 'min_data_in_leaf': 10, 'feature_fraction': 0.636593633990795, 'bagging_fraction': 0.8763178958731057, 'bagging_freq': 1, 'lambda_l1': 3.37372395608384, 'lambda_l2': 3.5508540427910322}

Best Suspension Trail:
{'num_leaves': 94, 'max_depth': 7, 'learning_rate': 0.053319349444258306, 'min_data_in_leaf': 11, 'feature_fraction': 0.7588234798454238, 'bagging_fraction': 0.9663253274432543, 'bagging_freq': 1, 'lambda_l1': 4.443326692620447, 'lambda_l2': 2.017979694136418}

Best Brake Balance Trail:
{'num_leaves': 52, 'max_depth': 14, 'learning_rate': 0.02166755029862027, 'min_data_in_leaf': 10, 'feature_fraction': 0.5221020637935397, 'bagging_fraction': 0.938010385446846, 'bagging_freq': 1, 'lambda_l1': 2.531857885425292, 'lambda_l2': 4.788757729441045}

In [ ]:
#Engine (Hyperparameters and X set) Check
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load both datasets
sim_df = pd.read_csv("simulator_data_newnames.csv")
prac_df = pd.read_csv("practice_data_newnamesV3_2.csv")

# Compute Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0


# Combine datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Features, target, and weights
X = df[['Engine', 'Grip', 'Humidity', 'Air Density', 'Altitude', 'Temperature', 'Inclines', 'Air Pressure' , 'Cornering']]
y = df["Avg Speed"]
weights = df["weight"]

# Define LightGBM dataset with weights
lgb_data = lgb.Dataset(X, label=y, weight=weights)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 46,
    'max_depth': 5,
    'learning_rate': 0.033632492994555285,
    'min_data_in_leaf': 10,
    'feature_fraction': 0.9292816238478901,
    'bagging_fraction': 0.6029213489987132,
    'bagging_freq': 1,
    'lambda_l1': 4.9435895890172095,
    'lambda_l2': 1.5835241201372334
    }

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("Engine_lgbm_model_R3.txt")

print("Model training complete and saved to 'Engine_lgbm_model_R3.txt'")


In [ ]:
#Differential (Hyperparameters and X set) check
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load both datasets
sim_df = pd.read_csv("simulator_data_newnames.csv")
prac_df = pd.read_csv("practice_data_newnamesV3_2.csv")

# Compute Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0


# Combine datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Features, target, and weights
X = df[['Differential', 'Engine', 'Cornering', 'Width', 'Inclines', 'Grip', 'Temperature', 'Air Density']]
y = df["Avg Speed"]
weights = df["weight"]

# Define LightGBM dataset with weights
lgb_data = lgb.Dataset(X, label=y, weight=weights)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 120,
    'max_depth': 4,
    'learning_rate': 0.06188740190651351,
    'min_data_in_leaf': 10,
    'feature_fraction': 0.8516027132719757,
    'bagging_fraction': 0.7665333052378347,
    'bagging_freq': 2,
    'lambda_l1': 4.842140375323366,
    'lambda_l2': 0.8572408057266877
    }


# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("Differential_lgbm_model_R3.txt")

print("Model training complete and saved to 'Differential_lgbm_model_R3.txt'")


In [ ]:
#Rear Wing (Hyperparameters and X set) check
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load both datasets
sim_df = pd.read_csv("simulator_data_newnames.csv")
prac_df = pd.read_csv("practice_data_newnamesV3_2.csv")

# Compute Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0


# Combine datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Features, target, and weights
X = df[['RearWing', 'Differential', 'Engine', 'Air Density', 'Cornering', 'Air Pressure' , 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Roughness']]
y = df["Avg Speed"]
weights = df["weight"]

# Define LightGBM dataset with weights
lgb_data = lgb.Dataset(X, label=y, weight=weights)


# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 234,
    'max_depth': 8,
    'learning_rate': 0.009001605992864,
    'min_data_in_leaf': 12,
    'feature_fraction': 0.7155500136774263,
    'bagging_fraction': 0.9782633351035283,
    'bagging_freq': 3,
    'lambda_l1': 4.0691300429761625,
    'lambda_l2': 0.4270601685583596}


# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("RearWing_lgbm_model_R3.txt")

print("Model training complete and saved to 'RearWing_lgbm_model_R3.txt'")


In [ ]:
#Front Wing (hyperparameters and X set)
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load both datasets
sim_df = pd.read_csv("simulator_data_newnames.csv")
prac_df = pd.read_csv("practice_data_newnamesV3_2.csv")

# Compute Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0


# Combine datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Features, target, and weights
X = df[['FrontWing', 'RearWing', 'Differential', 'Engine', 'Cornering', 'Air Pressure', 'Air Density', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Wind (Gusts)']]
y = df["Avg Speed"]
weights = df["weight"]

# Define LightGBM dataset with weights
lgb_data = lgb.Dataset(X, label=y, weight=weights)


# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 75,
    'max_depth': 15,
    'learning_rate': 0.008393206404444,
    'min_data_in_leaf': 10,
    'feature_fraction': 0.636593633990795,
    'bagging_fraction': 0.8763178958731057,
    'bagging_freq': 1,
    'lambda_l1': 3.37372395608384,
    'lambda_l2': 3.5508540427910322
    }



# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("FrontWing_lgbm_model_R3.txt")

print("Model training complete and saved to 'FrontWing_lgbm_model_R3.txt'")


In [ ]:
#Suspension
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load both datasets
sim_df = pd.read_csv("simulator_data_newnames.csv")
prac_df = pd.read_csv("practice_data_newnamesV3_2.csv")

# Compute Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0


# Combine datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Features, target, and weights
X = df[['Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine', 'Grip', 'Inclines', 'Cornering', 'Camber', 'Roughness', 'Width']]
y = df["Avg Speed"]
weights = df["weight"]

# Define LightGBM dataset with weights
lgb_data = lgb.Dataset(X, label=y, weight=weights)


# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 52,
    'max_depth': 14,
    'learning_rate': 0.02166755029862027,
    'min_data_in_leaf': 10,
    'feature_fraction': 0.5221020637935397,
    'bagging_fraction': 0.938010385446846,
    'bagging_freq': 1,
    'lambda_l1': 2.531857885425292,
    'lambda_l2': 4.788757729441045
    }

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("Suspension_lgbm_model_R3.txt")

print("Model training complete and saved to 'Suspension_lgbm_model_R3.txt'")


In [ ]:
#Brake Balance (hyperparameters and X set)
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load both datasets
sim_df = pd.read_csv("simulator_data_newnames.csv")
prac_df = pd.read_csv("practice_data_newnamesV3_2.csv")

# Compute Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0


# Combine datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Features, target, and weights
X = df[['BrakeBalance', 'Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine','Cornering', 'Width', 'Roughness', 'Temperature']]
y = df["Avg Speed"]
weights = df["weight"]

# Define LightGBM dataset with weights
lgb_data = lgb.Dataset(X, label=y, weight=weights)


# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 27,
    'max_depth': 11,
    'learning_rate': 0.18696777761444966,
    'min_data_in_leaf': 73,
    'feature_fraction': 0.964098272670725,
    'bagging_fraction': 0.8852821315081366,
    'bagging_freq': 10,
    'lambda_l1': 3.486399193949678,
    'lambda_l2': 3.7053586457221366
}

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("BrakeBalance_lgbm_model_R3.txt")

print("Model training complete and saved to 'BrakeBalance_lgbm_model_R3.txt'")


### Training and saving LGBM-Models for each Car-Parameter (Not SimPrac Combo)

In [ ]:
#Engine (Hyperparameters and X set)
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data_newnames.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df[['Engine', 'Grip', 'Humidity', 'Air Density', 'Altitude', 'Temperature', 'Inclines', 'Air Pressure' , 'Cornering']]
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 281,
    'max_depth': 11,
    'learning_rate': 0.04083240077095524,
    'min_data_in_leaf': 96,
    'feature_fraction': 0.9534759457760267,
    'bagging_fraction': 0.6637656869307713,
    'bagging_freq': 7,
    'lambda_l1': 0.48486124811843856,
    'lambda_l2': 1.0469435993282974
}

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("Engine_lgbm_model_R3.txt")

print("Model training complete and saved to 'Engine_lgbm_model_R3.txt'")


In [ ]:
#Differential (Hyperparameters and X set)
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data_newnames.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df[['Differential', 'Engine', 'Cornering', 'Width', 'Inclines', 'Grip', 'Temperature', 'Air Density']]
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 126,
    'max_depth': 8,
    'learning_rate': 0.05725939263903659,
    'min_data_in_leaf': 96,
    'feature_fraction': 0.9048274481436259,
    'bagging_fraction': 0.6871315558035545,
    'bagging_freq': 1,
    'lambda_l1': 2.9809700145880162,
    'lambda_l2': 2.893530172381254
    }


# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("Differential_lgbm_model_R3.txt")

print("Model training complete and saved to 'Differential_lgbm_model_R3.txt'")


In [ ]:
#Rear Wing (Hyperparameters and X set)
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data_newnames.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df[['RearWing', 'Differential', 'Engine', 'Air Density', 'Cornering', 'Air Pressure' , 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Roughness']]
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 41,
    'max_depth': 14,
    'learning_rate': 0.07379289439034513,
    'min_data_in_leaf': 14,
    'feature_fraction': 0.9805583874353474,
    'bagging_fraction': 0.757494844528942,
    'bagging_freq': 1,
    'lambda_l1': 3.3280185770912905,
    'lambda_l2': 4.668868493827121
    }

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("RearWing_lgbm_model_R3.txt")

print("Model training complete and saved to 'RearWing_lgbm_model_R3.txt'")


In [ ]:
#Front Wing (hyperparameters and X set)
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data_newnames.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df[['FrontWing', 'RearWing', 'Differential', 'Engine', 'Cornering', 'Air Pressure', 'Air Density', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Wind (Gusts)']]
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 228,
    'max_depth': 6,
    'learning_rate': 0.10991909073754202,
    'min_data_in_leaf': 94,
    'feature_fraction': 0.954670793846081,
    'bagging_fraction': 0.7659843231590522,
    'bagging_freq': 1, 'lambda_l1': 3.8616883301361553,
    'lambda_l2': 4.896984209036385
    }


# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("FrontWing_lgbm_model_R3.txt")

print("Model training complete and saved to 'FrontWing_lgbm_model_R3.txt'")


In [ ]:
#Suspension
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data_newnames.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df[['Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine', 'Grip', 'Inclines', 'Cornering', 'Camber', 'Roughness', 'Width']]
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'max_depth': 10,
    'learning_rate': 0.14051851191638579,
    'min_data_in_leaf': 75,
    'feature_fraction': 0.999888882811791,
    'bagging_fraction': 0.7009625088481276,
    'bagging_freq': 1,
    'lambda_l1': 4.8851994495913145,
    'lambda_l2': 3.6824821872767783
}

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("Suspension_lgbm_model_R3.txt")

print("Model training complete and saved to 'Suspension_lgbm_model_R3.txt'")


In [ ]:
#Brake Balance (hyperparameters and X set)
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data_newnames.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df[['BrakeBalance', 'Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine','Cornering', 'Width', 'Roughness', 'Temperature']]
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 27,
    'max_depth': 11,
    'learning_rate': 0.18696777761444966,
    'min_data_in_leaf': 73,
    'feature_fraction': 0.964098272670725,
    'bagging_fraction': 0.8852821315081366,
    'bagging_freq': 10,
    'lambda_l1': 3.486399193949678,
    'lambda_l2': 3.7053586457221366
}

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("BrakeBalance_lgbm_model_R3.txt")

print("Model training complete and saved to 'BrakeBalance_lgbm_model_R3.txt'")


Optimizing Models using Practice Data

In [ ]:
import pandas as pd
import lightgbm as lgb

# Model names
model_names = [
    "Engine_lgbm_model_R3.txt",
    "Differential_lgbm_model_R3.txt",
    "RearWing_lgbm_model_R3.txt",
    "FrontWing_lgbm_model_R3.txt",
    "Suspension_lgbm_model_R3.txt",
    "BrakeBalance_lgbm_model_R3.txt"
]

# Corresponding feature sets
feature_sets = {
    "Engine": ['Engine', 'Grip', 'Humidity', 'Air Density', 'Altitude', 'Temperature', 'Inclines', 'Air Pressure', 'Cornering'],
    "Differential": ['Differential', 'Engine', 'Cornering', 'Width', 'Inclines', 'Grip', 'Temperature', 'Air Density'],
    "RearWing": ['RearWing', 'Differential', 'Engine', 'Air Density', 'Cornering', 'Air Pressure', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Roughness'],
    "FrontWing": ['FrontWing', 'RearWing', 'Differential', 'Engine', 'Cornering', 'Air Pressure', 'Air Density', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Wind (Gusts)'],
    "Suspension": ['Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine', 'Grip', 'Inclines', 'Cornering', 'Camber', 'Roughness', 'Width'],
    "BrakeBalance": ['BrakeBalance', 'Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine', 'Cornering', 'Width', 'Roughness', 'Temperature']
}

# Load new data
df = pd.read_csv("practice_data_newnamesV3_1.csv")

# Compute Avg Speed for new data
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Target variable
y_new = df["Avg Speed"]

# Sample weight
sample_weight = [1000] * len(y_new)

# Training each model with its feature set
for model_file in model_names:
    model_key = model_file.split("_")[0]  # Extract the prefix e.g., "Engine"
    features = feature_sets[model_key]

    print(f"\nUpdating {model_file} with features: {features}")

    # Prepare feature matrix
    X_new = df[features]

    # Load existing model
    model = lgb.Booster(model_file=model_file)

    # Create dataset
    new_data = lgb.Dataset(X_new, label=y_new, weight=sample_weight)

    # Continue training
    updated_model = lgb.train(
        params={},
        train_set=new_data,
        init_model=model,
        num_boost_round=1000
    )

    # Save updated model
    updated_file = model_file.replace(".txt", "_updated.txt")
    updated_model.save_model(updated_file)

    print(f"Model saved to {updated_file}")

print("\nAll models updated successfully.")


### Sequentially optimizing Car Parameters.

In [ ]:
!pip install optuna

In [ ]:
import pandas as pd
import lightgbm as lgb
import optuna

# Ordered list of parameters to optimize
params_to_optimize = ["Engine", "Differential", "RearWing", "FrontWing", "Suspension", "BrakeBalance"]

# Corresponding feature sets for each model
feature_sets = {
    "Engine": ['Engine', 'Grip', 'Humidity', 'Air Density', 'Altitude', 'Temperature', 'Inclines', 'Air Pressure', 'Cornering'],
    "Differential": ['Differential', 'Engine', 'Cornering', 'Width', 'Inclines', 'Grip', 'Temperature', 'Air Density'],
    "RearWing": ['RearWing', 'Differential', 'Engine', 'Air Density', 'Cornering', 'Air Pressure', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Roughness'],
    "FrontWing": ['FrontWing', 'RearWing', 'Differential', 'Engine', 'Cornering', 'Air Pressure', 'Air Density', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Wind (Gusts)'],
    "Suspension": ['Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine', 'Grip', 'Inclines', 'Cornering', 'Camber', 'Roughness', 'Width'],
    "BrakeBalance": ['BrakeBalance', 'Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine', 'Cornering', 'Width', 'Roughness', 'Temperature']
}

# Load static track & weather data
base_data = pd.read_csv("track_weather_netherlands.csv")

# This will hold the best parameter values as we optimize them
optimized_params = {}

# Begin sequential optimization
for car_part in params_to_optimize:
    print(f"\n🔧 Optimizing {car_part}...")

    model_path = f"{car_part}_lgbm_model_R3.txt"
    model = lgb.Booster(model_file=model_path)
    features = feature_sets[car_part]

    def objective(trial):
        # Set the current parameter being optimized

        params = {
            "Engine": trial.suggest_int("Engine", 1, 500),
            "RearWing": trial.suggest_int("RearWing", 1, 500),
            "FrontWing": trial.suggest_int("FrontWing", 1, 500),
            "BrakeBalance": trial.suggest_int("BrakeBalance", 1, 500),
            "Suspension": trial.suggest_int("Suspension", 1, 500),
            "Differential": trial.suggest_int("Differential", 1, 500),}

        current_value = params[car_part]

        # Create a single row input with all needed features
        input_data = base_data.copy()

        # Add previously optimized params
        for param, value in optimized_params.items():
            input_data[param] = value

        # Add current trial value
        input_data[car_part] = current_value

        # If any missing feature, fill with 0 or a safe default
        for col in features:
            if col not in input_data.columns:
                input_data[col] = 0

        # Ensure correct order of features
        X = input_data[features]

        # Predict Avg Speed
        avg_speed = model.predict(X)[0]
        return avg_speed  # Optuna maximizes this

    # Run Optuna
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=500, show_progress_bar=True)

    best_value = study.best_params[car_part]
    optimized_params[car_part] = best_value
    print(f"✅ Best {car_part}: {best_value}")

# Final results
print("\n🎯 Final Optimized Parameters:")
for k, v in optimized_params.items():
    print(f"{k}: {v}")


In [ ]:
🎯 Final Optimized Parameters:
Engine: 77
Differential: 101
RearWing: 161
FrontWing: 80
Suspension: 41
BrakeBalance: 68

🎯 Final Optimized Parameters:
Engine: 45
Differential: 35
RearWing: 233
FrontWing: 72
Suspension: 160
BrakeBalance: 53

In [ ]:
import pandas as pd
import lightgbm as lgb
import optuna

# Ordered list of car parameters to optimize
params_to_optimize = [
    "Engine", "Differential", "RearWing",
    "FrontWing", "Suspension", "BrakeBalance"
]

# Feature sets used by each model for prediction
feature_sets = {
    "Engine": ['Engine', 'Grip', 'Humidity', 'Air Density', 'Altitude', 'Temperature', 'Inclines', 'Air Pressure', 'Cornering'],
    "Differential": ['Differential', 'Engine', 'Cornering', 'Width', 'Inclines', 'Grip', 'Temperature', 'Air Density'],
    "RearWing": ['RearWing', 'Differential', 'Engine', 'Air Density', 'Cornering', 'Air Pressure', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Roughness'],
    "FrontWing": ['FrontWing', 'RearWing', 'Differential', 'Engine', 'Cornering', 'Air Pressure', 'Air Density', 'Inclines', 'Wind (Avg. Speed)', 'Humidity', 'Wind (Gusts)'],
    "Suspension": ['Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine', 'Grip', 'Inclines', 'Cornering', 'Camber', 'Roughness', 'Width'],
    "BrakeBalance": ['BrakeBalance', 'Suspension', 'FrontWing', 'RearWing', 'Differential', 'Engine', 'Cornering', 'Width', 'Roughness', 'Temperature']
}

# Load static base data (track & weather)
base_data = pd.read_csv("track_weather_netherlands.csv")

# Dictionary to store optimized values
optimized_params = {}

# Sequential optimization loop
for car_part in params_to_optimize:
    print(f"\n🔧 Optimizing {car_part}...")

    # Load the pretrained LightGBM model
    model = lgb.Booster(model_file=f"{car_part}_lgbm_model_R3.txt")
    features = feature_sets[car_part]

    def objective(trial):
        # Clone base data for this trial
        input_data = base_data.copy()

        # Add fixed parameters (already optimized)
        for param, value in optimized_params.items():
            input_data[param] = value

        # Suggest value only for the current parameter
        trial_value = trial.suggest_int(car_part, 1, 500)
        input_data[car_part] = trial_value

        # Fill in missing feature columns with default (0)
        for feature in features:
            if feature not in input_data.columns:
                input_data[feature] = 0

        # Predict average speed
        X = input_data[features]
        avg_speed = model.predict(X)[0]
        return avg_speed

    # Run Optuna optimization for this parameter
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, show_progress_bar=True)

    # Save best value
    best_value = study.best_params[car_part]
    optimized_params[car_part] = best_value
    print(f"✅ Best {car_part}: {best_value}")

# Final results
print("\n🎯 Final Optimized Parameters:")
for k, v in optimized_params.items():
    print(f"{k}: {v}")


## AllInOne Optimization

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb

# Load the data
df = pd.read_csv("simulator_data.csv")

# Compute Avg Speed
df["Avg Speed"] = (df["Lap Distance"] / df["Lap Time"]) * 3600

# Features and target
X = df.drop(columns=["Avg Speed", "Lap Distance", "Lap Time"])
y = df["Avg Speed"]

# Define LightGBM dataset
lgb_data = lgb.Dataset(X, label=y)

# Parameters (tuned already)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 242,
    'max_depth': 9,
    'learning_rate': 0.0645474493988489,
    'min_data_in_leaf': 96,
    'feature_fraction': 0.9427185636727835,
    'bagging_fraction': 0.785711983453578,
    'bagging_freq': 2,
    'lambda_l1': 0.6006835353397709,
    'lambda_l2': 1.4905819957883624,
}

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("best_lgbm_model_R3.txt")

print("Model training complete and saved to 'best_lgbm_model_R3.txt'")


In [ ]:
#Directly trained LGBM-Model with Simulator_Data and Practice_data using optm. Hyperparameters
#Hyperparameters optm. for Simulator and Practice_data

import pandas as pd
import numpy as np
import lightgbm as lgb

# Load both datasets
sim_df = pd.read_csv("simulator_data.csv")
prac_df = pd.read_csv("practice_dataV3_2.csv")

# Compute Avg Speed
sim_df["Avg Speed"] = (sim_df["Lap Distance"] / sim_df["Lap Time"]) * 3600
prac_df["Avg Speed"] = (prac_df["Lap Distance"] / prac_df["Lap Time"]) * 3600

# Assign weights
sim_df["weight"] = 1.0
prac_df["weight"] = 250.0

# Make sure both have the same columns in the same order
required_cols = ["Lap Distance", "Lap Time", "Avg Speed", "weight", "Rear Wing", "Front Wing", "Engine", "Brake Balance", "Differential", "Suspension", "Cornering", "Inclines", "Camber", "Grip", "Altitude", "Roughness", "Width", "Temperature", "Humidity", "Wind (Avg. Speed)", "Wind (Gusts)", "Air Density", "Air Pressure"] + \
                [col for col in sim_df.columns if col not in ["Lap Distance", "Lap Time", "Avg Speed", "weight", "Rear Wing", "Front Wing", "Engine", "Brake Balance", "Differential", "Suspension", "Cornering", "Inclines", "Camber", "Grip", "Altitude", "Roughness", "Width", "Temperature", "Humidity", "Wind (Avg. Speed)", "Wind (Gusts)", "Air Density", "Air Pressure"]]

sim_df = sim_df[required_cols]
prac_df = prac_df[required_cols]

# Combine datasets
df = pd.concat([sim_df, prac_df], ignore_index=True)

# Features, target, and weights
X = df.drop(columns=["Avg Speed", "Lap Distance", "Lap Time", "weight"])
y = df["Avg Speed"]
weights = df["weight"]

# Define LightGBM dataset with weights
lgb_data = lgb.Dataset(X, label=y, weight=weights)

# Parameters (already tuned)
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 74,
    'max_depth': 11,
    'learning_rate': 0.028870630893942515,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.7399238619191133,
    'bagging_fraction': 0.9203471068533383,
    'bagging_freq': 6,
    'lambda_l1': 0.025758523534949052,
    'lambda_l2': 0.9073066525703107}

# Train on full dataset
model = lgb.train(params, lgb_data, num_boost_round=1000)

# Save the model to a file
model.save_model("sim_prac_lgbm_model_R3.txt")

print("Model training complete and saved to 'sim_prac_lgbm_model_R3.txt'")

In [ ]:
import pandas as pd
import lightgbm as lgb

# Load the original model
model = lgb.Booster(model_file="best_lgbm_model_R3.txt")
params = {
    'objective': 'regression',
    'metric': 'l2',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'num_leaves': 242,
    'max_depth': 9,
    'learning_rate': 0.0645474493988489,
    'min_data_in_leaf': 96,
    'feature_fraction': 0.9427185636727835,
    'bagging_fraction': 0.785711983453578,
    'bagging_freq': 2,
    'lambda_l1': 0.6006835353397709,
    'lambda_l2': 1.4905819957883624,
}
# Load the new data
new_df = pd.read_csv("practice_dataV3_1.csv")

# Compute Avg Speed for new data
new_df["Avg Speed"] = (new_df["Lap Distance"] / new_df["Lap Time"]) * 3600

# Prepare features and target
X_new = new_df.drop(columns=["Avg Speed", "Lap Distance", "Lap Time","Round", "Track", "Qualifying", "Stint", "Lap", "Fuel", "Tyre Remaining", "Tyre Choice"])
y_new = new_df["Avg Speed"]

# Assign higher weights (e.g., 10x more important)
sample_weight = [5] * len(y_new)

# Create new LightGBM Dataset
new_data = lgb.Dataset(X_new, label=y_new, weight=sample_weight)

# Continue training from previous model
model = lgb.train(
    params={},  # Empty here since model already knows them
    train_set=new_data,
    init_model=model,
    num_boost_round=1000,  # You can increase if needed
)

# Save updated model
model.save_model("best_lgbm_model_R3_updated.txt")

print("Model updated with new data and saved to 'best_lgbm_model_R3_updated.txt'")


In [ ]:
!pip install optuna


In [ ]:
import pandas as pd
import lightgbm as lgb
import optuna


# Load trained model
#model = lgb.Booster(model_file="best_lgbm_model_R3_updated.txt")
model = lgb.Booster(model_file="sim_prac_lgbm_model_R3.txt")

# Load track/weather data (single row)
track_weather = pd.read_csv("track_weather_netherlands.csv")

# Define the objective function
def objective(trial):
    # Suggest car parameters
    params = {
        "Engine": trial.suggest_int("Engine", 20, 60),
        "Rear Wing": trial.suggest_int("Rear Wing", 200, 300),
        "Front Wing": trial.suggest_int("Front Wing", 48, 88),
        "Brake Balance": trial.suggest_int("Brake Balance", 18, 118),
        "Suspension": trial.suggest_int("Suspension", 10, 110),
        "Differential": trial.suggest_int("Differential", 10, 100),
    }

#Engine: 45
#RearWing: 250
#FrontWing: 68
#BrakeBalance: 68
#Suspension: 60
#Differential: 35

    # Combine with static track/weather parameters
    input_data = pd.concat([track_weather, pd.DataFrame([params])], axis=1)

    # Predict average speed
    predicted_avg_speed = model.predict(input_data)[0]

    # We want to maximize speed
    return -predicted_avg_speed

# Create study
study = optuna.create_study(direction="minimize")

# Optimize
study.optimize(objective, n_trials=100)

# Output best results
best_trial = study.best_trial
print("\nBest Parameters:")
for key, value in best_trial.params.items():
    print(f"{key}: {value}")
print(f"Predicted Avg Speed: {-best_trial.value:.2f}")

## Race Strategy

In [ ]:
import math

# Define the tire parameters and their lap time formulas
def lap_time_super_soft(X):
    return (78.0565192575555) + 0.114973111111111 * X

def lap_time_soft(X):
    return (78.8737020908889) + 0.136290277777778 * X

def lap_time_medium(X):
    return (79.1300868242222) + 0.131553711111111 * X

def lap_time_hard(X):
    return (79.2600146591429) + 0.137075542857143 * X

# Tire data with their lifespan
tire_lifespan = {
    "super_soft": 12,
    "soft": 24,
    "medium": 30,
    "hard": 35
}

# Pit stop penalty
pit_stop_time = 30  # seconds

# Function to calculate the total race time for a given strategy
def calculate_race_time(laps, strategy):
    total_time = 0
    total_pit_stops = 0
    lap_index = 0
    lap_counter = 0

    while lap_counter < laps:
        tire, stint_laps = strategy[lap_index]

        # Ensure we don't exceed the total laps
        if lap_counter + stint_laps > laps:
            stint_laps = laps - lap_counter

        # Calculate the lap times for this stint
        lap_times = []
        for i in range(stint_laps):
            if tire == "super_soft":
                lap_times.append(lap_time_super_soft(i + 1))
            elif tire == "soft":
                lap_times.append(lap_time_soft(i + 1))
            elif tire == "medium":
                lap_times.append(lap_time_medium(i + 1))
            elif tire == "hard":
                lap_times.append(lap_time_hard(i + 1))

        total_time += sum(lap_times)  # Add the lap times of this stint
        lap_counter += stint_laps

        # If we are not at the last stint, account for a pit stop
        if lap_counter < laps:
            total_time += pit_stop_time  # Pit stop penalty
            total_pit_stops += 1

        lap_index += 1
        if lap_index >= len(strategy):
            break

    return total_time, total_pit_stops

# Function to generate possible strategies dynamically
def generate_strategies(laps):
    strategies = []
    tire_choices = ["super_soft", "soft", "medium", "hard"]

    # Generate strategies by breaking the laps into multiple stints
    for tire1 in tire_choices:
        for tire2 in tire_choices:
            for tire3 in tire_choices:
                for tire4 in tire_choices:
                  for tire5 in tire_choices:
                    strategy = []
                    remaining_laps = laps

                    # Create dynamic stints for each tire
                    for tire in [tire1, tire2, tire3, tire4, tire5]:
                    #for tire in [tire1, tire2, tire3, tire4]:
                    #for tire in [tire1, tire2, tire3]:
                    #for tire in [tire1, tire2]:
                        stint_laps = tire_lifespan[tire]

                        if remaining_laps > stint_laps:
                            strategy.append((tire, stint_laps))
                            remaining_laps -= stint_laps
                        else:
                            strategy.append((tire, remaining_laps))
                            break

                    if sum([stint[1] for stint in strategy]) == laps:
                        strategies.append(strategy)

    return strategies

# Function to find the best strategy
def optimize_strategy(laps):
    best_time = math.inf
    best_strategy = None

    strategies = generate_strategies(laps)

    for strategy in strategies:
        total_time, pit_stops = calculate_race_time(laps, strategy)
        if total_time < best_time:
            best_time = total_time
            best_strategy = strategy
            best_pit_stops = pit_stops

    return best_strategy, best_time, best_pit_stops


# Main function
if __name__ == "__main__":
    race_laps = 75
    best_strategy, best_time, total_pit_stops = optimize_strategy(race_laps)
    print(f"Best Strategy: {best_strategy}")
    print(f"Best Total Time: {best_time} seconds")
    print(f"Total Pit Stops: {total_pit_stops}")

# **Analytics for Race 4**

## Weights Calculation

In [ ]:
# rbf_weights.py  –  einmal pro Race ausführen
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis          # RBF-Distanz

META_COLS = [                                           # Ähnlichkeits-Features
    "Cornering", "Inclines", "Camber", "Grip",
    "Altitude", "Roughness", "Width",
    "Temperature", "Humidity",
    "Wind (Avg. Speed)", "Wind (Gusts)",
    "Air Density", "Air Pressure"
]

def build_dataset_with_rbf(sim_csv, practice_csv,
                           tau=2.0, practice_weight=5.0):
    """
    liest Sim- und Practice-CSV ein, berechnet RBF-Gewichte
    und gibt (DataFrame_all, weight_array) zurück.
    """
    df_sim = pd.read_csv(sim_csv)
    df_pr  = pd.read_csv(practice_csv)

    # -- RBF-Kernel auf den Meta-Features ------------------
    X_sim_meta = df_sim[META_COLS].values
    X_pr_meta  = df_pr[META_COLS].values
    centroid   = X_pr_meta.mean(axis=0)
    Sigma_inv  = np.linalg.inv(np.cov(X_sim_meta, rowvar=False))

    k_sim = np.exp([                                   # k_i = exp(-d_i / τ)
        -mahalanobis(x, centroid, Sigma_inv) / tau
        for x in X_sim_meta
    ])                                                 # :contentReference[oaicite:0]{index=0}

    w_sim = k_sim
    w_pr  = np.full(len(df_pr), practice_weight)

    weights = np.concatenate([w_sim, w_pr])
    df_all  = pd.concat([df_sim, df_pr], ignore_index=True)

    return df_all, weights


In [ ]:
# -------------------------------------------------------
# Schritt 0: Daten + Gewichte laden
# -------------------------------------------------------
from rbf_weights import build_dataset_with_rbf

df_all, weights = build_dataset_with_rbf(
    "simulator_data_newnames.csv",
    "practice_data.csv",      # deine echte 80-Lap-Datei
    tau=2.0,                  # Bandbreite feiner / gröber stellen
    practice_weight=5.0       # konstantes Gewicht der 80 Laps
)

# gemeinsame Zielvariable
df_all["Avg Speed"] = (df_all["Lap Distance"] / df_all["Lap Time"]) * 3600


In [ ]:
# Team members working on this code: Names..

# **Analytics for Race 5**

In [ ]:
# Team members working on this code: Names..

## **Debrief Race Calendar before Final Race**

*Write a longer text (200-500 words) reflecting on what were the main ideas you started the seminar with, how you improved your models to achieve better performance and what strategy and analytics you want to use for your final race during seminar day*

## **Analytics for Final Race**

In [ ]:
# Team members working on this code: Names..

## **References**
- Cite all references you need according to chair guidelines

Liu, Xuan; Shi, Savannah Wei; Teixeira, Thales; Wedel, Michel (2018): Video Content Marketing: The Making of Clips, Journal of Marketing, Vol. 82, 86-101.